# Load libraries

In [151]:
#Switch off startup messages
options(warn=0)
#Switch of summarize messages
options(dplyr.summarise.inform = FALSE)
#Limit preview output to 5 lines
options(repr.matrix.max.rows=3, repr.matrix.max.columns=5)
library(tidyverse)
library(ggpubr)
library(patchwork)
library(readxl)
library(ggh4x)
library(ggcorrplot)
library(ComplexUpset)
suppressPackageStartupMessages(library(ltm))
suppressPackageStartupMessages(library(LymphoSeq2))


Attaching package: ‘ComplexUpset’


The following object is masked from ‘package:UpSetR’:

    upset




# Load paths

In [8]:
analysis_path <- "~/zenodo"

## Metadata path

In [9]:
metadata_path <- str_c(analysis_path, "KSTME_study_metadata.csv", sep = "/")

## RNA-Seq reference path

In [19]:

hiv_fasta_path <- str_c(analysis_path, "hiv/GCF_000864765_protein.faa", sep = "/")
hhv8_fasta_path <- str_c(analysis_path, "kshv/GCF_000838265_protein.faa", sep = "/")
#hiv_gene_expression_path <- str_c(analysis_path, "rnaseq/hiv/salmon/kstme_hiv_genes_counts.tsv", sep = "/")
#hhv8_gene_expression_path <- str_c(analysis_path, "rnaseq/hhv8/salmon/kstme_hhv8_genes_counts.tsv", sep = "/")
#ilc_path <- str_c(analysis_path, "CRI_iAtlas_Portal_end_epi.csv", sep = "/") 
#gtex_nses_expression_path <- str_c(analysis_path, "cibersort_inputs/quant/gtex_nses/gene_tpm_2017-06-05_v8_skin_not_sun_exposed_suprapubic.gct", sep = "/")
#gtex_ses_expression_path <- str_c(analysis_path, "cibersort_inputs/quant/gtex_ses/gene_tpm_2017-06-05_v8_skin_sun_exposed_lower_leg.gct", sep = "/")
#lidenge_batchone_expression_path <- str_c(analysis_path, "rnaseq/human/cibersort_inputs/lidenge_batchone.tsv", sep = "/")
#lidenge_batchtwo_expression_path <- str_c(analysis_path, "rnaseq/human/cibersort_inputs/lidenge_batchtwo.tsv", sep = "/")
#lidenge_batchthree_expression_path <- str_c(analysis_path, "rnaseq/human/cibersort_inputs/lidenge_batchthree.tsv", sep = "/")
#lidenge_batchfour_expression_path <- str_c(analysis_path, "rnaseq/human/cibersort_inputs/lidenge_batchfour.tsv", sep = "/")
#tso_expression_path <- str_c(analysis_path, "rnaseq/human/cibersort_inputs/tso_epidemic.tsv", sep = "/")
#hippos_epidemic_expression <- str_c(analysis_path, "rnaseq/human/cibersort_inputs/hippos_epidemic_merged.tsv", sep = "/")
#hippos_endemic_expression <- str_c(analysis_path, "rnaseq/human/cibersort_inputs/hippos_endemic_merged.tsv", sep = "/")
# Targeted gene expression
#targerted_expression_path <- str_c(analysis_path, "nanostring/nanostring_counts.xlsx", sep = "/")

## RNA-Seq count path

In [25]:
hiv_gene_expression_path <- str_c(analysis_path, "rnaseq/kstme_hiv_genes_counts.tsv", sep = "/")
hhv8_gene_expression_path <- str_c(analysis_path, "rnaseq/kstme_hhv8_genes_counts.tsv", sep = "/")
cibersortx_results_path <- str_c(analysis_path, "cibersort", sep = "/")

## Nanostring input path

In [46]:
targerted_expression_path <- str_c(analysis_path, "nanostring/nanostring_counts.xlsx", sep = "/")

## AIRR inputs path

In [115]:
airr_seq_path <- str_c(analysis_path, "airrseq", sep = "/")
compairr_results_path <- str_c(airr_seq_path, "kstme_compairr_output.tsv", sep = "/")
antigen_db_path <- str_c(analysis_path, "tcr_database/known_tcr_database.tsv", sep = "/")
kstme_gliph_clusters_path <- str_c(airr_seq_path, "kstme_gliph_clusters.csv", sep = "/")
kstme_gliph_hlapred_path <- str_c(airr_seq_path, "kstme_gliph_hla_preds.csv", sep = "/")

## Output paths

In [38]:
figures_path <- str_c(analysis_path, "figures", sep = "/")

# Load metadata

In [13]:
study_metadata <- read_csv(metadata_path, show_col_types = FALSE)
study_metadata

cohort,patient_id,repertoire_id,tra_repertoire_id,trb_repertoire_id,rna_libraries,nanostring_libraries,gex_library,vdj_library,sc_library_count,⋯,DRB345allele1,DRB345allele2,timestamp,HIV,CD4_count,KSHV_plasma,phenotype,DPBallele2,ebv_status,bl_status
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,⋯,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
Hippos,008_001,008_001_A,NA,008_001_A,NA,001-A,NA,NA,NA,⋯,DRB3*01:01:02,DRB3*03:01:01,1/12/23,276908,127,0,Epidemic KS,NA,NA,NA
Hippos,008_001,008_001_B,008_001_B,008_001_B,008-001-B-12,001-B,NA,NA,NA,⋯,DRB3*01:01:02,DRB3*03:01:01,1/12/23,276908,127,0,Epidemic KS,NA,NA,NA
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
BL - Uganda,009_0249,MVQ194745A,NA,MVQ194745A,NA,NA,NA,NA,NA,⋯,DRB3*01:01:02,DRB3*02:02:01,NA,NA,NA,NA,BL,DPB1*105:01,Positive,Positive


# KSHV and HIV gene expression in KS tumors and M2 macrophages and CD8+ T-cells abundance in KS tumors

## Prepare HIV gene expression table

In [14]:
suppressWarnings({
hiv_dictionary <- read.table(hiv_fasta_path, sep="\n")$V1[grep(">", readLines(hiv_fasta_path))] |>
  as_tibble() |>
  separate(value, into = c("pid", "gene_name", "org"), sep = "\\s") |>
  mutate(pid = str_remove(pid, ">"),
    gene_name = if_else(gene_name == "Envelope", "Env", gene_name)) |>
    pull(gene_name)
names(hiv_dictionary) <- read.table(hiv_fasta_path, sep="\n")$V1[grep(">", readLines(hiv_fasta_path))] |>
  as_tibble() |>
  separate(value, into = c("pid", "gene_name", "org"), sep = "\\s") |>
  mutate(pid = str_remove(pid, ">"),
    gene_name = if_else(gene_name == "Envelope", "Env", gene_name)) |>
    pull(pid)
head(hiv_dictionary)})

NP_057849.4 NP_057850.1 NP_057851.1 NP_057852.2 NP_057853.1 NP_057854.1 
  "Gag-Pol" "Pr55(Gag)"       "Vif"       "Vpr"       "Tat"       "Rev"

In [17]:
# Prepare HIV gene expression table
hiv_expression_table <- read_tsv(hiv_gene_expression_path,
    show_col_types = FALSE) |>
  mutate(Name = str_remove_all(Name, "lcl\\|NC_001802.1_cds_|_\\d+$"),
    Name = hiv_dictionary[Name]) 
hiv_gene_names <- hiv_expression_table |>
  pull(Name)
hiv_expression_table

Name,008_001_B,008_001_C,008_001_I,008_003_C,008_004_D,008_005_C,008_006_B,008_006_C,008_007_B,⋯,008_029_C,008_030_B,008_030_C,008_032_B,008_034_B,008_037_B,008_122_B,008_140_B,008_175_B,008_211_D
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Vpr,0.0,0,109427.0,0,0,0,0,146631,108206,⋯,146745,36466.7,532094,77129.1,0,73775.5,0,821708,0,0
Tat,57184.7,0,55645.2,658517,392418,0,0,0,0,⋯,227184,142175.0,0,142841.0,0,20955.2,0,0,0,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
Gag-Pol,22543,1e+06,133181,60348.7,47980.9,365910,36689.5,26865.4,158001,⋯,142685,137332,117102,144536,16121,117851,0,0,0,222382


## Prepare KSHV gene expression table

In [20]:
# Prepare HHV8 gene dictionary
suppressWarnings({
hhv8_dictionary <- read.table(hhv8_fasta_path, sep="\n")$V1[grep(">", readLines(hhv8_fasta_path))] |>
  as_tibble() |>
  separate(value, into = c("pid", "gene_name", "org"), sep = "\\s") |>
  mutate(pid = str_remove(pid, ">"),
    gene_name = if_else(gene_name == "Envelope", "Env", gene_name)) |>
    pull(gene_name)
names(hhv8_dictionary) <- read.table(hhv8_fasta_path, sep="\n")$V1[grep(">", readLines(hhv8_fasta_path))] |>
  as_tibble() |>
  separate(value, into = c("pid", "gene_name", "org"), sep = "\\s") |>
  mutate(pid = str_remove(pid, ">"),
    gene_name = if_else(gene_name == "Envelope", "Env", gene_name)) |>
    pull(pid)
head(hhv8_dictionary)})

YP_001129350.1 YP_001129351.1 YP_001129352.1 YP_001129353.1 YP_001129354.1 
          "K1"         "ORF4"         "ORF6"         "ORF7"         "ORF8" 
YP_001129355.1 
        "ORF9"

In [24]:
# Prepare HHV8 gene expression data
hhv8_expression_table <- read_tsv(hhv8_gene_expression_path,
    show_col_types = FALSE) |>
  dplyr::mutate(Name = str_remove_all(Name, "lcl\\|NC_009333.1_cds_|_\\d+$"),
    Name = hhv8_dictionary[Name])
# Break up KSHV genes into groups based on stage of expression
#https://journals.plos.org/plospathogens/article?id=10.1371/journal.ppat.1001013#s5
#https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3894221/
kshv_groups <- c("vIRF-3" = "La", "ORF73" = "La", "K4.2" = "IE", "ORF45" = "IE",
  "ORF48" = "IE", "ORF50" = "IE", "K8" = "IE", "ORF70" = "E1", "ORF10" = "E1",
  "ORF56" = "E1", "ORF11" = "E1", "K3" = "E1", "ORF40" = "E1", "ORF59" = "E1",
  "K15" = "E1", "ORF54" = "E1", "ORF2" = "E2", "K1" = "E2", "K2" = "E2",
  "ORF46" = "E2", "ORF9" = "E2", "K5" = "E2", "K7" = "E2", "K6" = "E2",
  "vIRF-1" = "E2", "K14" = "E2", "vIRF-2" = "E2", "ORF49" = "E2", "ORF74" = "E2",
  "ORF7" = "E2", "ORF6" = "E2", "ORF44" = "E3", "ORF31" = "E3", "ORF19" = "E3",
  "ORF37" = "E3",  "ORF57" = "E3", "ORF17" = "E3", "ORF36" = "E3", "ORF29" = "E3",
  "ORF21" = "E3", "ORF61" = "E3", "ORF60" = "E3", "ORF66" = "E3", "ORF16" = "E3",
  "vIRF-4" = "E3", "ORF69" = "E3", "ORF23" = "L4", "ORF22" = "L4", "ORF25" = "L4",
  "ORF20" = "L4", "ORF26" = "L4", "ORF24" = "L4", "ORF27" = "L4", "ORF30" = "L4",
  "ORF28" = "L4", "ORF34" = "L4", "ORF18" = "L4", "ORF35" = "L4", "ORF63" = "L4",
  "ORF62" = "L4", "ORF65" = "L4", "ORF32" = "L4", "ORF38" = "L4", "ORF33" = "L4", 
  "ORF43" = "L4", "ORF64" = "L4", "ORF67" = "L4", "ORF68" = "L4", "ORF42" = "L5",
  "ORF4" = "L5", "ORF39" = "L5", "K8.1" = "L5", "ORF8" = "L5", "ORF47" = "L5",
  "ORF75" = "L5", "ORF58" = "L5", "ORF52" = "L5", "ORF53" = "L5", "ORF55" = "L5",
  "K12" = "La", "K4" = "E1", "K4.1" = "E1", "ORF17.5" = "E2", "ORF67A" = "L5",
  "ORF71" = "La", "ORF72" = "La" )
  hhv8_expression_table

Name,008_001_B,008_001_C,008_001_I,008_003_C,008_004_D,008_005_C,008_006_B,008_006_C,008_007_B,⋯,008_098_C,008_122_B,008_140_B,008_140_D,008_175_B,008_189_B,008_196_C,008_211_D,008_216_D,008_234_D
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ORF43,313.259,22.1425,1172.3,194.571,618.727,0,15.0339,69.0059,0.000,⋯,0.000,0,163.988,0.00,791.9940,28.4459,98.8281,0.000,296.225,640.235
ORF4,116.006,24.6127,0.0,648.511,1721.300,0,4563.1200,3313.0000,344.554,⋯,729.133,0,1569.200,2361.93,84.9846,1699.8700,278.5380,239.734,0.000,385.946
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
ORF48,661.551,154.753,0,0,988.613,0,0,54.8616,491.718,⋯,0,4012.77,976.673,148.605,1934.2,195.304,678.042,273.398,499.925,1770.07


## Prepare CibersortX results

In [26]:
# Load and format CibersortX results
cibersort_table <- list.files(cibersortx_results_path, pattern = "hippos",
    full.names = TRUE) |>
    map_df(~read_csv(.x, show_col_types = FALSE)) |>
    mutate(Mixture = str_remove(Mixture, "_quant"),
        Mixture = str_replace_all(Mixture, "-", "_"),
        Mixture = str_extract(Mixture, "008_\\d+_\\w")) |>
    dplyr::select(-`P-value`, -Correlation, -RMSE) |> 
    pivot_longer(c(-Mixture, -`Absolute score (sig.score)`), names_to = "cell_type", values_to = "score") |>
    mutate(percentage = (score*100)/`Absolute score (sig.score)`) |>
    pivot_wider(id_cols = Mixture, names_from = cell_type, values_from = percentage)
cibersort_table 

Mixture,B cells naive,B cells memory,Plasma cells,T cells CD8,T cells CD4 naive,T cells CD4 memory resting,T cells CD4 memory activated,T cells follicular helper,T cells regulatory (Tregs),⋯,Monocytes,Macrophages M0,Macrophages M1,Macrophages M2,Dendritic cells resting,Dendritic cells activated,Mast cells resting,Mast cells activated,Eosinophils,Neutrophils
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
008_061_C,7.912886,0,3.921065,19.70740,0,3.996909,1.182755,5.857817,0.5966646,⋯,1.163889,10.835613,11.778665,23.80936,1.952961,0,2.600103,0,0,0.0000000
008_098_B,6.596161,0,3.678067,21.50593,0,0.000000,9.657895,3.845360,0.9626662,⋯,0.000000,1.311762,6.150924,39.87146,0.000000,0,0.000000,0,0,0.2309073
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
008_037_B,7.057123,0,14.34114,18.42728,0,0,0,14.77712,0.3843394,⋯,0.5192566,8.19816,7.489275,20.11378,0,0,3.648959,0,0,0


## Reformat HIV expression table

In [27]:
# Generate HIV sample list table
# We will need this table to generate the final heatmap. 
# Since many samples do not show expression of HIV, this table will help
# generate a full table with 51 samples
sample_list <- cibersort_table |> 
  pull(Mixture)
hiv_gene_list <- hiv_expression_table |>
  pull(Name)

hiv_sample_table <- expand_grid(sample_list, hiv_gene_list) |>
  mutate(tpm = NULL) |>
  dplyr::rename(samples = sample_list, Name = hiv_gene_list)
sample_table <- cibersort_table |> 
  pull(Mixture) |> 
  as_tibble() |> 
  dplyr::rename(samples = value)
hiv_sample_table

samples,Name
<chr>,<chr>
008_061_C,Vpr
008_061_C,Tat
⋮,⋮
008_037_B,Gag-Pol


## Format RNA-Seq metadata

In [28]:
# Prep RNA-Seq metadata
rna_seq_metadata <-  study_metadata |> 
  filter(!is.na(rna_libraries)) |> 
  dplyr::select(cohort, repertoire_id, rna_libraries, tissue_type, phenotype)
rna_seq_metadata

cohort,repertoire_id,rna_libraries,tissue_type,phenotype
<chr>,<chr>,<chr>,<chr>,<chr>
Hippos,008_001_B,008-001-B-12,Tumor,Epidemic KS
Hippos,008_001_C,008-001-C-13,Tumor,Epidemic KS
⋮,⋮,⋮,⋮,⋮
TsoEtAl,SRR5787177,p23,Tumor,Epidemic KS


## Figure 1 : Epidemic and endemic KS tumors show both lytic and latent KSHV gene expression and contain abundant macrophages with M2 polarization and CD8+ T-cells. 

In [30]:
plot_order <- cibersort_table |> 
    left_join(rna_seq_metadata, by = c("Mixture" = "repertoire_id")) |>
    arrange(phenotype, `Macrophages M2`) |>
    pull(Mixture)

In [31]:
# Plot cibersortX panel
cibersort_plot <- cibersort_table |>
  pivot_longer(cols = `B cells naive`:Neutrophils, names_to = "cell_type", 
    values_to = "abundance" ) |> 
  left_join(rna_seq_metadata, by = c("Mixture" = "repertoire_id")) |>
  mutate(Mixture = factor(Mixture, levels = plot_order)) |>
  ggplot(aes(x = cell_type, y = Mixture, fill = abundance)) +
  geom_tile(color = "white", lwd = 0.25, linetype = 1) +
  labs(y = "TRB repertoire ID", x = "Cell type", fill = "Cell type\nabundance") +
  ggtitle("Immune cell-type\ncomposition") +
  scale_fill_viridis_b(option = "plasma") + 
  theme_classic(base_size = 20) +
  facet_grid(rows = vars(phenotype),scales = "free", space = "free", margins = F) +
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust = 1, face="bold"),  
    axis.title.x = element_blank(), 
    axis.text.y = element_blank(), axis.line.y = element_blank(), 
    axis.title.y = element_blank(), axis.ticks.y = element_blank(), 
    strip.background.x = element_blank(), 
    strip.background.y = element_rect(linewidth = 2, color = "black", linetype = 1), 
    strip.text.y.right = element_text(face = "bold"),
    strip.placement = "inside", plot.title = element_text(face = "bold")) 

In [32]:
# Plot HIV gene expression panel
hiv_plot <- hiv_expression_table |>
  pivot_longer(-Name, names_to = "samples",
    values_to = "tpm") |>
  full_join(hiv_sample_table) |> 
  left_join(rna_seq_metadata, by = c("samples" = "repertoire_id")) |>
  mutate(tpm = if_else(tpm == 0, NA_integer_, tpm),
    samples = factor(samples, levels = plot_order)) |>
  ggplot(aes(x = Name, y = samples, fill = tpm)) +
  geom_tile(color = "white", lwd = 0.25, linetype = 1) +
  labs(x = "Gene", fill = "TPM") +
  ggtitle("HIV gene\nexpression") +
  scale_fill_viridis_c(option = "viridis", 
    trans = scales::pseudo_log_trans(sigma = 0.001),
    breaks = c(0, 10, 100, 1000, 10000, 100000, 1000000)) +
  theme_classic(base_size = 20) +
  facet_grid(rows = vars(phenotype), scales = "free", space = "free", 
    margins = F) +
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust = 1, face="bold"), 
    axis.title.x = element_blank(), legend.position = "none", 
    axis.text.y = element_blank(), axis.line.y = element_blank(), 
    axis.title.y = element_blank(), axis.ticks.y = element_blank(),  
    strip.background = element_blank(), strip.text.y = element_blank(),
    strip.placement = "inside", plot.title = element_text(face = "bold")) 

Joining with `by = join_by(Name, samples)`


In [33]:
# Plot KSHV gene expression
kshv_plot <- hhv8_expression_table |>
  pivot_longer(col = `008_001_B`:`008_234_D`, names_to = "samples",
    values_to = "tpm") |>
  mutate(tpm = if_else(tpm == 0, NA_integer_, tpm)) |>
  right_join(sample_table) |> 
  left_join(rna_seq_metadata, by = c("samples" = "repertoire_id")) |>
  mutate(stage = factor(kshv_groups[Name], 
    levels =c("La", "IE", "E1", "E2", "E3", "L4", "L5", NA)),
    samples = factor(samples, levels = plot_order)) |>
  ggplot(aes(x = Name, y = samples, fill = tpm)) +
  geom_tile(color = "white", lwd = 0.25, linetype = 1) +
  labs(x = "Gene", fill = "TPM", y = "Sample ID") +
  ggtitle("KSHV gene\nexpression") +
  scale_fill_viridis_c(option = "viridis", 
    trans = scales::pseudo_log_trans(sigma = 0.001),
    breaks = c(0, 10, 100, 1000, 10000, 100000, 1000000)) +
  theme_classic(base_size = 20) +
  facet_grid(rows = vars(phenotype), cols = vars(stage), scales = "free", 
    space = "free", margins = F)  +
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust = 1, face="bold"),
    axis.text.y = element_text(face = "bold"),
    axis.title.x = element_blank(),panel.spacing.x = unit(0.25, "lines"),
    strip.text.y = element_blank(), strip.text.x = element_text(face="bold"),
    strip.placement = "inside", axis.title.y = element_text(face = "bold"),
    plot.title = element_text(face = "bold")) 

Joining with `by = join_by(samples)`


In [36]:
# Plot cell type boxplot
cohort_palette <- c("Epidemic KS" = "#e41a1c", "Endemic KS" = "#377eb8")
cibersort_boxplots <- cibersort_table |>
  pivot_longer(`B cells naive`:Neutrophils, names_to = "cell_type", 
    values_to = "abundance") |>
  filter(cell_type %in% c("Macrophages M2", 
    "T cells CD4 memory activated", "T cells CD4 memory resting",
    "T cells CD8", "T cells follicular helper")) |>
  left_join(rna_seq_metadata, by = c("Mixture" = "repertoire_id")) |>
  mutate(cell_type = case_when(cell_type == "T cells CD4 memory activated" ~ "T cells CD4\nmemory activated",
    cell_type == "T cells CD4 memory resting" ~ "T cells CD4\nmemory resting",
    cell_type == "T cells follicular helper" ~ "T cells\nfollicular helper",
    TRUE ~ cell_type)) |>
  ggboxplot(x = "phenotype", y = "abundance", color = "phenotype", 
    facet.by = "cell_type", nrow = 1) |>
  ggpar(xlab = "Cohort", ylab = "Abundance", legend.title = "Cohort",
    ggtheme = theme_classic(base_size = 20), ylim = c(0, 60),
    legend = "right", palette = cohort_palette,
    panel.labs.font = list(face = "bold")) +
  stat_compare_means(comparisons = list(c("Endemic KS", "Epidemic KS")),
    size = 5, label = "p.signif") +
  theme(strip.text.x = element_text(face="bold"),
    axis.text.x = element_blank(), axis.ticks.x = element_blank(),
    axis.title = element_text(face = "bold"))

In [37]:
## Plot correlation scatter plot for M2 macrophages and CD8 T-cells
tcell_mac_corr_plot <- cibersort_table |>
    left_join(rna_seq_metadata, by = c("Mixture" = "repertoire_id")) |>
    ggplot(aes(x = `Macrophages M2`, y = `T cells CD8`, color = phenotype)) +
    geom_point() +
    labs(ylab = "T cells CD8 (%)",
        xlab = "Macrophage M2 (%)") +
    geom_smooth(method = "glm", aes(fill=phenotype)) +
    ylim(0, 50) + 
    stat_cor(p.accuracy = 0.001, r.accuracy = 0.01, size = 8)+
    scale_color_manual(values = cohort_palette) + 
    scale_fill_manual(values = cohort_palette) + 
    theme_classic(base_size = 20) +
    theme(axis.title = element_text(face = "bold"),
        legend.position = "none")

In [43]:
# Assemble all panels of Figure 1
figure_one_layout <- "
1111111111111111112233333
1111111111111111112233333
1111111111111111112233333
1111111111111111112233333
4444444444444444444455555"
suppressWarnings({
figure_one <- kshv_plot +  hiv_plot +
  cibersort_plot  +  cibersort_boxplots + tcell_mac_corr_plot +
  plot_layout(design = figure_one_layout, guides = "collect",
    widths =c(9, 1, 2.5, 10, 2.5)) +
  plot_annotation(tag_levels = 'A') & theme(text = element_text('NimbusSan'),
    plot.tag = element_text(size = 36,face = "bold"))
ggsave(str_c(figures_path, "pdf", "Figure_one.pdf", sep = "/"), figure_one, 
  limitsize = FALSE, width = 30, height = 24, device = cairo_pdf, family = "Arial Unicode MS")
ggsave(str_c(figures_path, "svg", "Figure_one.svg", sep = "/"), figure_one,
  limitsize = FALSE, width = 30, height = 24, device = "svg")
ggsave(str_c(figures_path, "png", "Figure_one.png", sep = "/"), figure_one,
  limitsize = FALSE, width = 30, height = 24, device = "png")})

`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'


## Supplementary figure 1: Targeted gene expression profiling of epidemic and endemic KS tumors.

In [47]:
phenotype_table <- study_metadata |> 
    filter(cohort == "Hippos") |>
    select(patient_id, phenotype) |>
    mutate(phenotype = replace_na(phenotype, "Epidemic KS")) |>
    distinct()

In [50]:
nanostring_matrix <- read_excel(targerted_expression_path, sheet=1) |>
  pivot_longer(cols = c(-`Probe Name`, -Function), names_to = "sample", values_to = "counts")
nanostring_meta <- read_excel(targerted_expression_path, sheet=2) |>
  pivot_longer(cols = `001-B`:His9DP, names_to = "sample", values_to = "tissue_type") |> 
  select(sample, tissue_type) |> 
  distinct()

nanostring_matrix <- left_join(nanostring_matrix, nanostring_meta, by = "sample")
my_breaks <- c(0, 0.01, 0.1, 1, 10, 100, 1000, 10000)

In [51]:
# Nanostring heatmap
nanostring_plot <- nanostring_matrix |> 
  dplyr::filter(!is.na(tissue_type) & sample != "KS + HC") |>
  mutate(tissue_type = case_when(str_detect(sample, "A$") ~ "NAT",
        str_detect(sample, "^His") ~ "Histology",
        TRUE ~ "Tumor"),
    Function = case_when(Function == "Tumor proliferation" ~ "Proliferation",
      Function == "T-cell proliferation" ~ "T-cell",
      Function == "Tumor inhibition" ~ "M1 Macrophage",
      TRUE ~ Function),
    Function = factor(Function, levels = c("T-cell", "CD4 T-cell", "CD8 T-cell", 
      "T-reg", "T-cell inhibition", "NK cell", 
      "Macrophage", "M1 Macrophage", "M2 Macrophage", 
      "Proliferation"))) |>
  filter(tissue_type != "Histology") |>
  ggplot(aes(x = sample, y = `Probe Name`, fill = counts)) +
  geom_tile(aes(width=0.9, height=0.9), color = "black", linewidth=0.1) +
  labs(x = "Samples", y = "Probe", fill = "Counts", tag = "B") +
  scale_fill_viridis_c(breaks = my_breaks, labels = my_breaks, 
                       trans = scales::pseudo_log_trans(sigma = 0.001)) +
  theme_classic(base_size = 26) +
  facet_grid(cols = vars(tissue_type), rows = vars(Function), scales = "free", space = "free", margins = F) +
  theme(axis.text.x = element_blank(),  axis.title.x = element_blank(),
    axis.line.y = element_blank(), axis.title.y = element_blank(), 
    axis.text.y = element_text(face = "bold"),
    strip.placement = "inside", axis.ticks.y = element_blank(),
    strip.text.y = element_blank(), strip.text.x = element_text(face = "bold"),
    legend.position = "bottom",
    legend.key.size = unit(2, "cm"),
    text = element_text(family = "NimbusSan"),
    plot.tag = element_text(size = 36,face = "bold")) 

In [52]:
# Nanostring log fold change
nanostringfc_plot <- nanostring_matrix |> 
  dplyr::filter(is.na(tissue_type) & sample != "KS + HC") |>
  mutate(tissue_type = if_else(sample == "KS + HC", "Lesion\nvs\nNormal/HC", "FC"),
    Function = case_when(Function == "Tumor proliferation" ~ "Proliferation",
      Function == "T-cell proliferation" ~ "T-cell",
      Function == "Tumor inhibition" ~ "M1 Macrophage",
      TRUE ~ Function),
    Function = factor(Function, levels = c("T-cell", "CD4 T-cell", "CD8 T-cell", 
      "T-reg", "T-cell inhibition", "NK cell", 
      "Macrophage", "M1 Macrophage", "M2 Macrophage", 
      "Proliferation"))) |>
  ggplot(aes(x = sample, y = `Probe Name`, fill = counts)) +
  geom_tile(aes(width=0.9, height=0.9), color = "black", linewidth=0.1) +
  labs(x = "Samples", y = "Probe", fill = "Log2FC") +
  scale_fill_viridis_b(option = "plasma") +
  theme_classic(base_size = 26) +
  facet_grid(cols = vars(tissue_type), rows = vars(Function), scales = "free", space = "free", margins = F) +
  theme(axis.text.x = element_blank(),  axis.title.x = element_blank(),
    axis.text.y = element_blank(),  
    axis.line.y = element_blank(), axis.title.y = element_blank(), 
    strip.placement = "inside", axis.ticks.y = element_blank(),
    strip.text.x = element_text(face = "bold"),
    strip.text.y = element_text(angle = 0, face = "bold"), legend.position = "bottom",
    text = element_text(family = "NimbusSan")) 

In [53]:
# Nanostring viral expression
targeted_viral_expression <- read_excel(targerted_expression_path, sheet = 4)
viral_expression_plot <- targeted_viral_expression  |>
    mutate(Target = case_match(Target,
      "HHV4" ~ "EBV",
      "HHV5" ~ "CMV",
      "HHV8" ~ "KSHV",
      .default = Target)) |>
    pivot_longer(c(-Target, -`Probe Name`), names_to = "sample", values_to = "l2fc") |>
    mutate(sample = case_when(sample == "8020" ~ "008_020",
        sample == "030-B" ~ "008_030", sample == "028-B" ~ "008_028",
        sample == "024-B" ~ "008_024", sample == "013-B" ~ "008_013",
        sample == "008-C" ~ "008_008", sample == "001-B" ~ "008_001",
        TRUE ~ str_replace(sample, "-", "_")),
      sample = factor(sample, levels = c("008_001", "008_008", "008_013",
          "008_020", "008_024", "008_028", "008_030", "008_036", "008_037",
          "008_052", "008_059", "008_061", "008_062", "008_075", "008_085",
          "008_088", "008_092", "008_093", "008_097", "008_099", "008_101",
          "008_106"))) |>
    left_join(phenotype_table, by = c("sample" = "patient_id")) |>
    arrange(sample) |>
    mutate(Target = factor(Target, levels = c("Endothelial", "EBV", "CMV", "KSHV", "HIV1", "HIV2"))) |>
    ggplot(aes(y = `sample`, x = `Probe Name`, fill = l2fc)) +
    geom_tile(color = "white", lwd = 0.5, linetype = 1) +
    labs(x = "Probe name", y = "Sample", fill = "Log2FC", tag = "A") +
    #scale_fill_gradientn(colors = rev(c('#b2182b','#d6604d','#d1e5f0'))) +
    scale_fill_viridis_b(breaks = c(2,4,6,8,10)) +
    theme_classic(base_size = 20) +
    facet_grid(cols = vars(Target), rows = vars(phenotype), 
      scales = "free", space = "free", margins = F) +
    theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust = 1, face = "bold", family = 'NimbusSan'),
        axis.text.y = element_text(face = "bold", family = 'NimbusSan'),
         strip.text.x = element_text(face="bold", family = 'NimbusSan'),
         strip.text.y = element_text(face = "bold", angle = 0, family = 'NimbusSan'),
        axis.title.x = element_blank(), 
        axis.line.y = element_blank(), legend.key.size = unit(1, "cm"),
        axis.title.y = element_blank(), axis.ticks.y = element_blank(),
        plot.tag = element_text(size = 36,face = "bold")) 


In [54]:
plot_design <- "11111111111111111111111111111
                22222222222222222222222222223"
supplementary_figure_one <- (viral_expression_plot +labs(tag = "A"))+ 
  (nanostring_plot + labs(tag = "B")) +  
  nanostringfc_plot +
  plot_layout(design = plot_design) & theme(plot.tag = element_text(size = 36,face = "bold"))

ggsave(str_c(figures_path, "pdf", "Supplementary_figure_one.pdf", sep = "/"), supplementary_figure_one, 
  limitsize = FALSE, width = 25, height = 30, device = cairo_pdf, family = "Arial Unicode MS")
ggsave(str_c(figures_path, "svg", "Supplementary_figure_one.svg", sep = "/"), supplementary_figure_one,
  limitsize = FALSE, width = 25, height = 30, device = "svg")
ggsave(str_c(figures_path, "png", "Supplementary_figure_one.png", sep = "/"), supplementary_figure_one,
  limitsize = FALSE, width = 25, height = 30, device = "png")


## Figure 2: KS tumors show an enrichment of M2 macrophages and CD8+ T-cells in comparison to control skin samples.

In [64]:
# Load cibersort results from public KS datasets
readCiberSort <- function(path) {
    study = basename(path)
    cib_table <- read_csv(path, show_col_types = FALSE) |>
        mutate(Mixture = str_remove(Mixture, "_quant"),
            cohort = study,
            cohort = case_when(cohort == "gtex_nes.csv" ~ "Non sun-exposed skin",
                cohort == "gtex_ses.csv" ~ "Sun-exposed skin",
                str_detect(cohort, "lid") ~ "Lidenge et al.",
                str_detect(cohort, "tso") ~ "Tso et al."))
    return(cib_table)
}


public_cibersort_table <- list.files(cibersortx_results_path,
        pattern = "lid|tso|gtex", full.names = TRUE) |>
    map_df(readCiberSort)  |>
    select(-`P-value`, -Correlation, -RMSE) |> 
    pivot_longer(c(-Mixture, -cohort,  -`Absolute score (sig.score)`),
        names_to = "cell_type", values_to = "score") |>
    mutate(percentage = (score*100)/`Absolute score (sig.score)`) |>
    pivot_wider(id_cols = c(Mixture, cohort), names_from = cell_type,
        values_from = percentage)

In [66]:
figure_two_heatmap <- public_cibersort_table |>
  pivot_longer(cols = `B cells naive`:Neutrophils, names_to = "cell_type", 
    values_to = "abundance" ) |>
  mutate(Mixture = if_else(cohort == "Lidenge et al.", 
    str_extract(Mixture, "SRR\\d+$"), Mixture)) |>
  left_join(rna_seq_metadata, by = c("Mixture" = "repertoire_id")) |>
  mutate(phenotype = case_when(cohort.x %in% c("Non sun-exposed skin", "Sun-exposed skin") & is.na(tissue_type) ~ "Control", 
    cohort.x == "Lidenge et al." & tissue_type == "Normal" ~ "Con-\ntrol",
    tissue_type == "Tumor" & phenotype == "Endemic KS" ~ "Ende-\nmic KS", 
    cohort.x == "Tso et al." & tissue_type == "Tumor" & phenotype == "Epidemic KS" ~ "Epide-\nmic KS", 
    tissue_type == "Tumor" & phenotype == "Epidemic KS" ~ "Epidemic KS",
    TRUE ~ tissue_type),
    cohort.x = factor(cohort.x, levels = c("Lidenge et al.", "Tso et al.", "Non sun-exposed skin", "Sun-exposed skin")),
    phenotype = factor(phenotype, levels = c("Con-\ntrol", "Control", "NAT", "Ende-\nmic KS", "Endemic KS", "Epide-\nmic KS", "Epidemic KS"))) |>
  ggplot(aes(y = cell_type, x = Mixture, fill = abundance)) +
  geom_tile(color = "white", lwd = 0.5, linetype = 1) +
  labs(x = "Sample", y = "Cell type", fill = "Cell type\nabundance") +
  scale_fill_viridis_b(option = "plasma") + 
  theme_classic(base_size = 20) +
  facet_nested(cols = vars(cohort.x, phenotype),scales = "free", space = "free", margins = F) +
  theme(axis.text.x = element_blank(), strip.background = element_blank(),
    axis.title.x = element_blank(), strip.text = element_text(face = "bold"),
    axis.text.y = element_text(face = "bold"), axis.line = element_blank(), 
    axis.ticks = element_blank(),
    text = element_text(family = 'NimbusSan')) 

In [67]:
supp_cohort_palatte <- c("Epidemic KS - NAT" = "#fb8072", 
  "Epidemic KS - Tumor" = "#e41a1c", "Endemic KS - NAT" = "#80b1d3",
  "Endemic KS - Tumor" = "#377eb8", "Control" = "#67a628")


figure_two_corrplot <- public_cibersort_table |>
    mutate(Mixture = if_else(cohort == "Lidenge et al.", 
    str_extract(Mixture, "SRR\\d+$"), Mixture)) |>
    left_join(rna_seq_metadata, by = c("Mixture" = "repertoire_id")) |>
    mutate(phenotype = case_when(cohort.x %in% c("Non sun-exposed skin", "Sun-exposed skin") & is.na(tissue_type) ~ "Control", 
    cohort.x == "Lidenge et al." & tissue_type == "Normal" ~ "Control",
    tissue_type == "Tumor" & phenotype == "Endemic KS" ~ "Endemic KS - Tumor", 
    tissue_type == "Tumor" & phenotype == "Epidemic KS" ~ "Epidemic KS - Tumor", 
    tissue_type == "NAT" & phenotype == "Epidemic KS" ~ "Epidemic KS - NAT", 
    tissue_type == "NAT" & phenotype == "Endemic KS" ~ "Endemic KS - NAT"),
    cohort.x = factor(cohort.x, levels = c("Lidenge et al.", "Tso et al.", "Non sun-exposed skin", "Sun-exposed skin")),
    phenotype = factor(phenotype)) |>
    ggscatter(x = "Macrophages M2", y = "T cells CD8", color = "phenotype", 
        facet.by = "cohort.x", nrow = 1,
    add = "reg.line")  + 
    stat_cor(aes(color = phenotype), p.accuracy = 0.001, r.accuracy = 0.01, size = 6) +
    theme_classic(base_size = 20) + 
    labs(y = "T cells CD8 (%)", x = "Macrophages M2 (%)", color = "Tissue type") +
    ylim(0,50) +
    scale_color_manual(values = supp_cohort_palatte) +
    scale_fill_manual(values = supp_cohort_palatte) +
    theme(strip.text.x = element_text(face="bold"),
      axis.title = element_text(face = "bold"),
      legend.position = "right")

In [68]:
figure_two <- figure_two_heatmap / figure_two_corrplot + 
  plot_annotation(tag_levels = "A") & theme(plot.tag = element_text(size = 36,face = "bold"))
ggsave(str_c(figures_path, "pdf", "Figure_two.pdf", sep = "/"), figure_two, 
  limitsize = FALSE, width = 30, height = 14, device = cairo_pdf, family = "Arial Unicode MS")
ggsave(str_c(figures_path, "svg", "Figure_two.svg", sep = "/"), figure_two,
  limitsize = FALSE, width = 30, height = 14, device = "svg")
ggsave(str_c(figures_path, "png", "Figure_two.png", sep = "/"), figure_two,
  limitsize = FALSE, width = 30, height = 14, device = "png")

# KS TIL repertoires are diverse and poorly correlated with plasma KSHV viral load at study entry and 1-year survival

For easy of reproducibility, we will load checkpoints generated from the raw data. To generate these checkpoints please run the nextflow pipelines packaged with the GitHub repository

In [63]:
# Load checkpoints when avaiable
load(str_c(airr_seq_path, "kstme_paper_tcr_tables.rda", sep = "/"))
load(str_c(airr_seq_path, "kstme_paper_raw_tables.rda", sep = "/"))

In [69]:
annotated_nprod_table <- study_metadata |>
  inner_join(study_nprod_table, by = c("trb_repertoire_id" = "repertoire_id"))

In [70]:
# Tumors per individual

study_metadata |> 
    inner_join(annotated_summary, by = c("repertoire_id" = "repertoire_id")) |>
    filter(total_sequences >= 1000 & tissue_type == "Tumor") |>
    group_by(patient_id) |>
    summarize(number_of_repertoires = length(unique(trb_repertoire_id))) |>
    pull(number_of_repertoires) |>
    summary()

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  1.000   1.000   1.000   1.871   3.000   7.000 

## Figure 3: Diversity of T-cells infiltrating endemic KS tumors is comparable to the diversity observed in TIL from EBV-associated Burkitt lymphoma.

In [71]:
# Subset tumor and NAT samples from the Renyi table and merge with metadata
til_normal_renyi <- study_renyi_table |> 
    rename(trb_repertoire_id = repertoire_id) |>
    pivot_longer(cols = `0`:`Inf`, names_to = "alpha", 
        values_to = "renyi_number") |>
    inner_join(study_metadata, by = "trb_repertoire_id") |>
    filter(cohort %in% c("Hippos", "BL - Ghana", "BL - Uganda") & 
        tissue_type %in% c("NAT", "Tumor") & 
        (visit_code == "V01" | is.na(visit_code)) & !is.na(trb_repertoire_id)) |>
    mutate(cohort = case_when(
        cohort == "Hippos"  ~  str_c(phenotype, tissue_type, sep = " - "),
        cohort != "Hippos" ~ cohort),
        cohort = factor(cohort,  
            levels = c("Endemic KS - NAT", "Endemic KS - Tumor",
                "Epidemic KS - NAT", "Epidemic KS - Tumor", 
                "BL - Ghana", "BL - Uganda")),
        alpha = str_replace(alpha, "Inf", "\u221e")) 

In [72]:
comparisons <- list(c("Endemic KS - Tumor", "Epidemic KS - Tumor"),
   c("Epidemic KS - NAT", "Epidemic KS - Tumor"), 
   c("Endemic KS - Tumor", "BL - Ghana"),
  c("Epidemic KS - Tumor", "BL - Ghana"),
  c("Epidemic KS - Tumor", "BL - Uganda"))
comparisons_clin <- list(c("Endemic KS", "Epidemic KS"))
cohort_palette <- c("Epidemic KS - NAT" = "#fb8072", 
  "Epidemic KS - Tumor" = "#e41a1c", "Endemic KS - NAT" = "#80b1d3",
  "Endemic KS - Tumor" = "#377eb8", "BL - Uganda" = "#a65628",
  "BL - Ghana" = "#fdb462")

alpha_zero <- til_normal_renyi |>
  filter(alpha == "0") |>
  ggboxplot(x = "cohort", y = "renyi_number", color = "cohort") |>
  ggpar(xlab = FALSE, ylab = "Species richness\n(\u03b1 = 0)", legend.title = "Cohort",
    ggtheme = theme_classic(base_size = 24),
    legend = "none", palette = cohort_palette) +
  rremove("x.text") +
  stat_compare_means(comparisons = list(c("Endemic KS - Tumor", "Epidemic KS - Tumor"),
   c("Endemic KS - Tumor", "BL - Ghana"),
  c("Epidemic KS - Tumor", "BL - Ghana"),
  c("Epidemic KS - Tumor", "BL - Uganda")), 
  hide.ns = TRUE, size = 5, label = "p.signif", show.legend = T)
alpha_one <- til_normal_renyi |>
  filter(alpha == "1") |>
  ggboxplot(x = "cohort", y = "renyi_number", color = "cohort") |>
  ggpar(xlab = FALSE, ylab = "Shannon entropy\n(\u03b1 = 1)", legend.title = "Cohort",
    ggtheme = theme_classic(base_size = 24),
    legend = "none", palette = cohort_palette) + 
  rremove("x.text") +
  stat_compare_means(comparisons = list(c("Endemic KS - Tumor", "Epidemic KS - Tumor"),
   c("Endemic KS - Tumor", "BL - Ghana"),
  c("Epidemic KS - Tumor", "BL - Uganda")), hide.ns = TRUE, size = 5, label = "p.signif", show.legend = T)
alpha_two <- til_normal_renyi |>
  filter(alpha == "2") |>
  ggboxplot(x = "cohort", y = "renyi_number", color = "cohort") |>
  ggpar(xlab = "Cohort", ylab = "Simpson's diversity\n(\u03b1 = 2)", legend.title = "Cohort",
    ggtheme = theme_classic(base_size = 24), 
    legend = "none", palette = cohort_palette, x.text.angle = 45) +
  stat_compare_means(comparisons = list(c("Endemic KS - Tumor", "Epidemic KS - Tumor"),
   c("Epidemic KS - NAT", "Epidemic KS - Tumor"), 
   c("Endemic KS - Tumor", "BL - Ghana"),
  c("Epidemic KS - Tumor", "BL - Uganda")), 
  hide.ns = TRUE, size = 5, label = "p.signif", show.legend = T)
alpha_inf <- til_normal_renyi |>
  filter(alpha == "\u221e") |>
  ggboxplot(x = "cohort", y = "renyi_number", color = "cohort") |>
  ggpar(xlab = "Cohort", ylab = "Berger-Parker index\n(\u03b1 = \u221e)", legend.title = "Cohort",
    ggtheme = theme_classic(base_size = 24), 
    legend = "right", palette = cohort_palette, x.text.angle = 45) +
  stat_compare_means(comparisons = list(
   c("Epidemic KS - NAT", "Epidemic KS - Tumor"), 
  c("Epidemic KS - Tumor", "BL - Uganda")), 
  hide.ns = TRUE, size = 5, label = "p.signif", show.legend = T)
renyi_plot <- ggline(til_normal_renyi, x = "alpha", y = "renyi_number",
    color = "cohort", add = "mean_se") |> 
  ggpar(xlab = "\u03b1", ylab = "Renyi entropy", legend.title = "Cohort",
    ggtheme = theme_classic(base_size = 24), 
    legend = "none", palette = cohort_palette)
figure_three_layout <- "
1122
1122
3344
3344
5555
5555"
figure_three <- alpha_zero + alpha_one + alpha_two + 
  alpha_inf + renyi_plot + 
  plot_layout(design = figure_three_layout, guides = "collect") +
  plot_annotation(tag_levels = 'A') & theme(text = element_text('NimbusSan'),
    plot.tag = element_text(size = 36,face = "bold"))
ggsave(str_c(figures_path, "pdf", "Figure_three.pdf", sep = "/"), figure_three, width = 16, 
    height = 18, device = cairo_pdf, family = "Arial Unicode MS")
ggsave(str_c(figures_path, "svg", "Figure_three.svg", sep = "/"), figure_three, width = 16, 
    height = 18)
ggsave(str_c(figures_path, "png", "Figure_three.png", sep = "/"), figure_three, width = 16, 
    height = 18)

## Supplementary Figure 2: Plasma KSHV and HIV viral load and CD4+ T-cell count are not correlated with T-cell repertoire diversity in KS tumors

In [73]:
# Plot KSHV viral load and CD4+ T-cell counts
plot_titer_tables <- study_metadata |>
  filter(trb_repertoire_id %in% til_normal_renyi$repertoire_id) |>
  select(patient_id, visit_code, hiv_status, ks_status,
    HIV, CD4_count, KSHV_plasma) |>
  distinct()
kshv_titer_plot <- plot_titer_tables |>
  filter(!is.na(KSHV_plasma)) |>
  mutate(cohort = if_else(hiv_status == "Positive", "Epidemic KS",
    "Endemic KS")) |>
  ggboxplot(x = "cohort", y = "KSHV_plasma", color = "cohort", 
    add = "jitter") |>
  ggpar(xlab = FALSE, ylab = "Plasma KSHV titer\n(copies/mL)", 
    legend.title = "Cohort",
    ggtheme = theme_classic(base_size = 24),
    legend = "none", palette = c("Epidemic KS" = "#e41a1c",
      "Endemic KS" = "#377eb8")) +
  stat_compare_means(comparisons = comparisons_clin, hide.ns = TRUE, size = 5, label = "p.signif", show.legend = T)
cd4_plot <- plot_titer_tables |>
  filter(!is.na(CD4_count)) |>
  mutate(cohort = if_else(hiv_status == "Positive", "Epidemic KS",
    "Endemic KS")) |>
  ggboxplot(x = "cohort", y = "CD4_count", color = "cohort", 
    add = "jitter") |>
  ggpar(xlab = FALSE, ylab = str_c("CD4+ count\n(cells/\u03BC", "L)", sep = ""), 
    legend.title = "Cohort",
    ggtheme = theme_classic(base_size = 24),
    legend = "none", palette = c("Epidemic KS" = "#e41a1c",
      "Endemic KS" = "#377eb8")) +
  geom_hline(yintercept = 200, linetype = 2) +
  stat_compare_means(comparisons = comparisons_clin, hide.ns = TRUE, size = 5, label = "p.signif", show.legend = T)


In [76]:
# Calculate the correlation between clinical metrics and sequencing metrics
correlation_data_frame <- study_metadata |>
    filter(cohort == "Hippos" & tissue_type %in% c("NAT", "Tumor") & visit_code == "V01") |>
    filter(!is.na(HIV) & !is.na(CD4_count) & !is.na(KSHV_plasma)) |>
    select(patient_id, repertoire_id, HIV, CD4_count, KSHV_plasma, phenotype) |> 
    inner_join(study_renyi_table, by = "repertoire_id") |>
    dplyr::select(repertoire_id, HIV:`Inf`, -phenotype)
corr_row_names <- correlation_data_frame |>
    pull(repertoire_id)
correlation_data_frame <- correlation_data_frame |>
    select(-repertoire_id) |>
    as.data.frame()
rownames(correlation_data_frame) <- corr_row_names
colnames(correlation_data_frame) <- c("HIV\nVL", "CD4+\ncount", "KSHV\nPlasma\nVL", "0", "0.25", "0.5", "1", "2", "4", "8", "16", "32", "64", "\u221e")
correlation_matrix <- round(cor(correlation_data_frame), 1)

In [79]:
correlation_plot <- ggcorrplot(correlation_matrix,  outline.col = "white",  lab = TRUE) +
  theme_classic() +
  theme(axis.text = element_text(size = 12, face = "bold"), axis.title = element_blank(),
    legend.key.size = unit(1, "cm"), text = element_text(family = "NimbusSan"))

In [80]:
supp_figure_two_layout <- "
1122
3333
3333"

supplementary_figure_two <- kshv_titer_plot + 
  cd4_plot  + 
  correlation_plot +
  plot_layout(design = supp_figure_two_layout) +
  plot_annotation(tag_levels = 'A') & theme(text = element_text('NimbusSan'),
    plot.tag = element_text(size = 36,face = "bold"))



ggsave(str_c(figures_path, "pdf", "Supplementary_figure_two.pdf", sep = "/"), supplementary_figure_two, 
  limitsize = FALSE, width = 14, height = 15, device = cairo_pdf, family = "Arial Unicode MS")
ggsave(str_c(figures_path, "svg", "Supplementary_figure_two.pdf", sep = "/"), supplementary_figure_two,
  limitsize = FALSE, width = 14, height = 15, device = "svg")
ggsave(str_c(figures_path, "png", "Supplementary_figure_two.pdf", sep = "/"), supplementary_figure_two,
  limitsize = FALSE, width = 14, height = 15, device = "png")

## Determining the correlation between response and sequencing metrics

In [87]:
clin_correlation_data_frame <- study_metadata |>
    filter(cohort == "Hippos" & tissue_type %in% c("NAT", "Tumor") & visit_code == "V01") |>
    dplyr::select(patient_id, repertoire_id, phenotype, response) |> 
    inner_join(study_renyi_table, by = "repertoire_id") |>
    filter(!is.na(response)) |>
    mutate(response = if_else(response == "CR" | response == "PR", 1, 0)) 
# Endemic KS
endemic_zero <- biserial.cor(clin_correlation_data_frame |> 
                             filter(phenotype == "Endemic KS") |> 
                             pull(`0`), 
                             clin_correlation_data_frame |> 
                             filter(phenotype == "Endemic KS") |> 
                             pull(response), 
                             use = c("all.obs"), level = 2)
endemic_one <- biserial.cor(clin_correlation_data_frame |> 
                            filter(phenotype == "Endemic KS") |> 
                            pull(`1`),
                            clin_correlation_data_frame |>
                            filter(phenotype == "Endemic KS") |> 
                            pull(response), 
                            use = c("all.obs"), level = 2)
endemic_two <- biserial.cor(clin_correlation_data_frame |> 
                            filter(phenotype == "Endemic KS") |> 
                            pull(`2`), 
                            clin_correlation_data_frame |> 
                            filter(phenotype == "Endemic KS") |> 
                            pull(response), 
                            use = c("all.obs"), level = 2)
endemic_inf <- biserial.cor(clin_correlation_data_frame |> 
                            filter(phenotype == "Endemic KS") |> 
                            pull(`Inf`), 
                            clin_correlation_data_frame |> 
                            filter(phenotype == "Endemic KS") |> 
                            pull(response), 
                            use = c("all.obs"), level = 2)
endemic_biserial_table <- tibble(Renyi = c("0", "1", "2", "Inf"), Corr = c(endemic_zero, endemic_one, endemic_two, endemic_inf))
endemic_biserial_table

Renyi,Corr
<chr>,<dbl>
0,0.1659725
1,0.1301313
⋮,⋮
Inf,0.09401167


In [95]:
# epidemic KS
epidemic_zero <- biserial.cor(clin_correlation_data_frame |> 
                             filter(phenotype == "Epidemic KS") |> 
                             pull(`0`), 
                             clin_correlation_data_frame |> 
                             filter(phenotype == "Epidemic KS") |> 
                             pull(response), 
                             use = c("all.obs"), level = 2)
epidemic_one <- biserial.cor(clin_correlation_data_frame |> 
                            filter(phenotype == "Epidemic KS") |> 
                            pull(`1`),
                            clin_correlation_data_frame |>
                            filter(phenotype == "Epidemic KS") |> 
                            pull(response), 
                            use = c("all.obs"), level = 2)
epidemic_two <- biserial.cor(clin_correlation_data_frame |> 
                            filter(phenotype == "Epidemic KS") |> 
                            pull(`2`), 
                            clin_correlation_data_frame |> 
                            filter(phenotype == "Epidemic KS") |> 
                            pull(response), 
                            use = c("all.obs"), level = 2)
epidemic_inf <- biserial.cor(clin_correlation_data_frame |> 
                            filter(phenotype == "Epidemic KS") |> 
                            pull(`Inf`), 
                            clin_correlation_data_frame |> 
                            filter(phenotype == "Epidemic KS") |> 
                            pull(response), 
                            use = c("all.obs"), level = 2)
epidemic_biserial_table <- tibble(Renyi = c("0", "1", "2", "Inf"), Corr = c(epidemic_zero, epidemic_one, epidemic_two, epidemic_inf))
epidemic_biserial_table

Renyi,Corr
<chr>,<dbl>
0,0.1637493
1,0.1602944
⋮,⋮
Inf,0.1648721


In [96]:
# All KS
all_zero <- biserial.cor(clin_correlation_data_frame |> 
                             pull(`0`), 
                             clin_correlation_data_frame |> 
                             pull(response), 
                             use = c("all.obs"), level = 2)
all_one <- biserial.cor(clin_correlation_data_frame |> 
                            pull(`1`),
                            clin_correlation_data_frame |>
                            pull(response), 
                            use = c("all.obs"), level = 2)
all_two <- biserial.cor(clin_correlation_data_frame |> 
                            pull(`2`), 
                            clin_correlation_data_frame |> 
                            pull(response), 
                            use = c("all.obs"), level = 2)
all_inf <- biserial.cor(clin_correlation_data_frame |> 
                            pull(`Inf`), 
                            clin_correlation_data_frame |> 
                            pull(response), 
                            use = c("all.obs"), level = 2)
all_biserial_table <- tibble(Renyi = c("0", "1", "2", "Inf"), Corr = c(all_zero, all_one, all_two, all_inf))
all_biserial_table

Renyi,Corr
<chr>,<dbl>
0,0.2418664
1,0.2298005
⋮,⋮
Inf,0.1804144


# TIL repertoires of KS tumors are largely private to each individual

First we will understand the sharing of individual TCR sequences across repertoires

In [ ]:
# Create a dictionary of repertoires to repertoire filenames
rep_dict <- study_metadata |>
  filter(!is.na(trb_repertoire_id)) |>
  pull(repertoire_id)
names(rep_dict) <- study_metadata |>
  filter(!is.na(trb_repertoire_id)) |>
  pull(trb_repertoire_id)
# Create a dictionary of patient IDs to phenotypes
ptid_dict <- study_metadata |>
  filter(!is.na(patient_id)) |>
  pull(phenotype)
names(ptid_dict) <- study_metadata |>
  filter(!is.na(patient_id)) |>
  pull(patient_id)
ptid_dict["008_265"] <- "Epidemic KS"
# Load compairr results
compairr_table <- read_tsv(compairr_results_path, show_col_types = FALSE) |>
  dplyr::rename(rep_one = `#`) 
comp_names <- compairr_table |>
  pull(rep_one)
compairr_matrix <- compairr_table |>
  dplyr::select(-rep_one) |>
  as.matrix() 
rownames(compairr_matrix) <- comp_names
# Create a lower triangle matrix and reformat it to a table for tidy manipulations
compairr_matrix[lower.tri(compairr_matrix, diag = FALSE)] <- NA 
compairr_table <- compairr_matrix |>
  as_tibble(rownames = "rep_one") |>
  pivot_longer(-rep_one, names_to = "rep_two", values_to = "overlap") |>
  filter(str_detect(rep_one, "008_\\d+") & str_detect(rep_two, "008_\\d+") & 
    !is.na(overlap) & 
    str_extract(rep_one, "[A-Z]+$") %in% c("A","B", "C", "D") &
    str_extract(rep_two, "[A-Z]+$") %in% c("A","B", "C", "D")) |>
  mutate(rep_one = rep_dict[rep_one], rep_two = rep_dict[rep_two],
    sample_group = if_else(str_extract(rep_one, "008_\\d+") ==
      str_extract(rep_two, "008_\\d+"), "Intra-subject", "Inter-subject"),
    tumor_group = case_when(
      str_extract(rep_one, "[A-Z]+$") == "A" &
        str_extract(rep_two, "[A-Z]+$") == "A" ~ "NAT - NAT",
      str_extract(rep_one, "[A-Z]+$") == "A" &
        str_extract(rep_two, "[A-Z]+$") %in% c("B", "C", "D") ~ "NAT - Tumor",
      str_extract(rep_two, "[A-Z]+$") == "A" &
        str_extract(rep_one, "[A-Z]+$") %in% c("B", "C", "D") ~ "NAT - Tumor",
      str_extract(rep_two, "[A-Z]+$") %in% c("B", "C", "D") &
        str_extract(rep_one, "[A-Z]+$") %in% c("B", "C", "D") ~ "Tumor - Tumor"),
    ks_group = str_c(ptid_dict[str_extract(rep_one, "008_\\d+")],
      ptid_dict[str_extract(rep_two, "008_\\d+")], sep = " - "),
    ks_group = str_replace(ks_group, "Epidemic KS - Endemic KS",
      "Endemic KS - Epidemic KS")) |>
  filter(!(sample_group == "Inter-subject" & tumor_group == "NAT - Tumor") &
    !(rep_one == rep_two))
# Generate data table for 4 representative samples used in the alluvial plots
alluvial_data <- study_annotated_nprod_table |>
  filter(patient_id %in% c("008_057", "008_061", "008_021", "008_048") & 
    visit_code == "V01") |>
  productiveSeq()


## Annotate TCRs with known anitgenic specifcity as determined by comparison with VDJdb and McPAS-TCR DB

In [110]:
antigen_db <- read_tsv(antigen_db_path, show_col_types = FALSE)

In [112]:
# Match all TCRs from the study AIRR-Seq data to sequences listed in the public databases accounting for HLA type of the individuals showing the TCR
hla_matched_tcr <- annotated_nprod_table |>
    mutate(visit_code = if_else(is.na(visit_code), "V01", visit_code)) |>
    filter(cohort %in% c("Hippos") & visit_code == "V01" &
        tissue_type %in% c("NAT", "Tumor")) |>
    inner_join(antigen_db, by = c("junction_aa" = "trb_cdr3_aa"), 
        relationship = "many-to-many") |>
    dplyr::select(patient_id, repertoire_id, junction, junction_aa, epitope, pathology,
        antigen, mhc_allele, aAllele1:DRB345allele2) |> 
    pivot_longer(cols = aAllele1:DRB345allele2, names_to = "allele",
        values_to = "mhc_value") |> 
    mutate(mhc_allele = str_remove(mhc_allele, "HLA-"),
        hla_matched = if_else(mhc_allele == mhc_value |
            mhc_allele == str_extract(mhc_value, "\\w+\\*\\d+:\\d+") |
            mhc_allele == str_extract(mhc_value, "\\w+\\*\\d+"), TRUE, FALSE)) |>
    filter(hla_matched) |> 
    dplyr::select(patient_id:junction_aa, epitope:antigen) |>
    distinct() |> 
    group_by(patient_id, repertoire_id, junction, junction_aa) |>
    summarise(npath = length(unique(pathology)),
        pathology = str_c(unique(pathology), collapse = ";")) |>
    ungroup() |>
    mutate(pathology = if_else(npath > 1, "Multi-pathogen", pathology)) |>
    dplyr::select(patient_id, repertoire_id, junction, junction_aa, pathology) |>
    distinct() |>
    mutate(pathology = recode(pathology, 
        `HomoSapiens` = "Other", "Homo sapiens" = "Other", 
        "SelaginellaMoellendorffii" = "Other", "TriticumAestivum" = "Other", 
        "synthetic" = "Other"))

In [113]:
path_annotated_nprod_table <- annotated_nprod_table |> 
    mutate(visit_code = if_else(is.na(visit_code), "V01", visit_code)) |>
    filter(cohort %in% c("Hippos") & visit_code == "V01" &
        tissue_type %in% c("NAT", "Tumor")) |>
    left_join(hla_matched_tcr, by = c("patient_id", "repertoire_id", "junction",
        "junction_aa")) |>
    dplyr::select(cohort, patient_id, repertoire_id, tissue_type, phenotype, pathology, junction_aa, duplicate_frequency) |>
    mutate(pathology = str_replace_na(pathology, "Unknown"),
        phenotype = if_else(cohort == "ARKS", "ARKS", phenotype)) |>
    group_by(cohort, repertoire_id, pathology, phenotype, tissue_type) |>
    summarize(frequency = sum(duplicate_frequency)) |>
    ungroup() |>
    mutate(cohort = if_else(cohort == "Hippos", 
            str_c(phenotype, tissue_type, sep = " - "), "ARKS")) 

cohort,repertoire_id,pathology,phenotype,tissue_type,frequency
<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
Epidemic KS - NAT,008_001_A,Unknown,Epidemic KS,NAT,1
Epidemic KS - Tumor,008_001_B,Unknown,Epidemic KS,Tumor,1
⋮,⋮,⋮,⋮,⋮,⋮
Epidemic KS - Tumor,008_265_D,Unknown,Epidemic KS,Tumor,0.9972152


## Format GLIPH results and annotate with antigenic specificity

In [116]:
kstme_gliph_cluster_pred_table <- read_csv(kstme_gliph_clusters_path,
        show_col_types = FALSE) 
kstme_gliph_hla_pred_table <- read_csv(kstme_gliph_hlapred_path, 
    show_col_types = FALSE)

In [117]:
# Filter out high confidence GLIPH groups with a pattern length of >2 where, 
# the pattern has at 3 unique CDR3s associated with it, at least 3 PTIDs 
# associated with it and with a vb_score <= 0.01
kstme_hc_gliph_cluster_table <- kstme_gliph_cluster_pred_table |>
  dplyr::select(pattern, vb_score, hla_score, expansion_score, TcRb, Sample, V, J,
    Freq) |>
  separate(Sample, into = c("repertoire_id", "cohort"), sep = ":") |>
  mutate(patient_id = str_extract(repertoire_id, "008_\\d+")) |>
  group_by(pattern) |>
  mutate(number_of_unique_cdr = length(unique(TcRb)),
    number_of_unique_ptid = length(unique(patient_id))) |>
  ungroup() |>
  filter(vb_score <= 0.01 & number_of_unique_cdr >= 3 &
    number_of_unique_ptid >= 3 & pattern != "single" & nchar(pattern) > 2)

In [118]:
# Filter out HLA predictions for high confidence GLIPH groups identified in the
# previous step
kstme_hc_gliph_hla_table <- kstme_gliph_hla_pred_table |>
  dplyr::rename(sig_allele = `'HLA allele with lowest Fisher Score'`) |>
  mutate(pattern = str_remove(pattern, "^[lg]")) |>
  filter(pattern %in% kstme_hc_gliph_cluster_table$pattern & Pvalue <= 0.05) |>
  dplyr::select(pattern, sig_allele, Pvalue)

In [119]:
# Merge GLIPH clusters with significant HLA association
kstme_high_confidence_gliph_groups <- kstme_hc_gliph_cluster_table |>
  left_join(kstme_hc_gliph_hla_table, by = "pattern",
    relationship = "many-to-many") 

In [121]:
# Annotate GLIPH groups with antigenic specificity based on the CDR3 sequences
# they contain
path_annotated_gliph_clusters <- hla_matched_tcr |>
  distinct() |>
  inner_join(kstme_high_confidence_gliph_groups, by = c("repertoire_id", "junction_aa" = "TcRb"),
    relationship = "many-to-many") |>
  dplyr::select(repertoire_id, pathology, pattern) |>
  distinct()

In [122]:
 kstme_ck_gliph_table <- kstme_high_confidence_gliph_groups |>
  dplyr::select(repertoire_id, pattern, TcRb, V, J) |>
  left_join(path_annotated_gliph_clusters, by = c("repertoire_id", "pattern"), relationship = "many-to-many") |>
  distinct() |>
  filter(is.na(pathology)) |>
  mutate(pathology = if_else(is.na(pathology), "Clustered\nUnknown", pathology)) |>
  group_by(TcRb, V, J) |>
  mutate(npath = length(unique(pathology)),
    pathology = str_c(unique(pathology), collapse =";")) |>
  ungroup() |>
  mutate(pathology = if_else(npath > 1, "Multi-pathogen", pathology)) 

In [123]:
# Annotate GLIPH groups by antigenic specificity of CDR3s grouped within
kstme_path_annotated_gliph_groups <- kstme_high_confidence_gliph_groups |>
  dplyr::select(repertoire_id, pattern, TcRb, V, J) |>
  left_join(path_annotated_gliph_clusters, by = c("repertoire_id", "pattern"), relationship = "many-to-many") |>
  distinct() |>
  mutate(pathology = if_else(is.na(pathology), "Clustered\nUnknown", pathology)) |>
  group_by(TcRb, V, J) |>
  mutate(npath = length(unique(pathology)),
    pathology = str_c(unique(pathology), collapse =";")) |>
  ungroup() |>
  mutate(pathology = if_else(npath > 1, "Multi-pathogen", pathology)) 

In [124]:
# Annotate GLIPH groups as "Multi-pathogen" when they contain TCRs mapped to different antigens
kstme_gliph_nprod_table <- annotated_nprod_table |>
    mutate(visit_code = if_else(is.na(visit_code), "V01", visit_code)) |>
    filter(cohort %in% c("Hippos") & visit_code == "V01" &
        tissue_type %in% c("NAT", "Tumor")) |>
    left_join(kstme_path_annotated_gliph_groups, by = c("repertoire_id", 
        "junction_aa" = "TcRb", "v_call" = "V",
        "j_call" = "J"), relationship = "many-to-many") |>
    distinct() |>
    dplyr::select(cohort, patient_id, repertoire_id, tissue_type, phenotype, pathology, junction_aa, v_call, j_call, duplicate_frequency) |>
    mutate(pathology = if_else(is.na(pathology), "Unclustered\nUnknown", pathology),
        phenotype = if_else(is.na(phenotype) , "Epidemic KS", phenotype)) |>
    group_by(junction_aa, v_call, j_call ) |>
    mutate(npath = length(unique(pathology)),
        pathology = str_c(unique(pathology), collapse =";")) |>
    ungroup() |>
    mutate(pathology = if_else(npath > 1, "Multi-pathogen", pathology)) |>
    distinct()

In [129]:
# Generate GLIPH pattern associated with each TCR annotated with antigen groups. (Note: very similar to the previous block, we just keep the pattern information)
kstme_gliph_tpack_table <- annotated_nprod_table |>
    mutate(visit_code = if_else(is.na(visit_code), "V01", visit_code)) |>
    filter(cohort %in% c("Hippos") & visit_code == "V01" &
        tissue_type %in% c("NAT", "Tumor")) |>
    left_join(kstme_path_annotated_gliph_groups, by = c("repertoire_id", 
        "junction_aa" = "TcRb", "v_call" = "V",
        "j_call" = "J"), relationship = "many-to-many") |>
    distinct() |>
    dplyr::select(cohort, patient_id, repertoire_id, tissue_type, phenotype, pathology, 
        junction_aa, v_call, j_call, duplicate_count, duplicate_frequency, pattern) |>
    mutate(pathology = if_else(is.na(pathology), "Unclustered\nUnknown", pathology),
        phenotype = if_else(is.na(phenotype) , "Epidemic KS", phenotype)) |>
    group_by(junction_aa, v_call, j_call ) |>
    mutate(npath = length(unique(pathology)),
        pathology = str_c(unique(pathology), collapse =";")) |>
    ungroup() |>
    mutate(pathology = if_else(npath > 1, "Multi-pathogen", pathology)) |>
    distinct()

In [130]:
# Summarize sequence label table to pathogen level table
path_annotated_gliph_nprod_table <- kstme_gliph_nprod_table |>
    group_by(cohort, repertoire_id, pathology, phenotype, tissue_type) |>
    summarize(frequency = sum(duplicate_frequency)) |>
    ungroup() 

In [131]:
# Extract just the "clustered unknown" subgroup
kstme_clustered_unknown_table <- kstme_gliph_tpack_table |>
    filter(pathology == "Clustered\nUnknown")

In [133]:
# Extract a table of GLIPH pattern from each repertoire
kstme_gliph_pattern_table <- kstme_clustered_unknown_table |>
    dplyr::select(repertoire_id, pattern) |>
    filter(!is.na(repertoire_id) & !is.na(pattern)) |>
    distinct()

## Summarizing GLIPH mapping of sequences

In [134]:
intra_tumor_sharing_count <- kstme_clustered_unknown_table |>
    filter(!str_detect(repertoire_id, "008_\\d+_A")) |>
    group_by(pattern, patient_id) |>
    summarise(nreps = length(unique(repertoire_id))) |>
    filter(nreps > 1) |>
    pull(pattern) |>
    unique() |>
    length()
str_c("Number of specificity groups shared by at least two tumors derieved from spatially distinct sites from the same individual", intra_tumor_sharing_count, sep = " : ")

[1] "Number of specificity groups shared by at least two tumors derieved from spatially distinct sites from the same individual : 26509"

In [135]:
inter_tumor_sharing_count <- kstme_clustered_unknown_table |>
    filter(!str_detect(repertoire_id, "008_\\d+_A")) |>
    group_by(pattern) |>
    summarise(nptids = length(unique(patient_id))) |>
    filter(nptids > 1) |>
    pull(pattern) |>
    unique() |>
    length()
str_c("Number of specificity groups shared by at least two tumors derieved from spatially distinct sites from the different individuals", inter_tumor_sharing_count, sep = " : ")

[1] "Number of specificity groups shared by at least two tumors derieved from spatially distinct sites from the different individuals : 42747"

In [136]:
inter_by_cond_tumor_sharing_count <- kstme_clustered_unknown_table |>
    filter(!str_detect(repertoire_id, "008_\\d+_A")) |>
    group_by(pattern,phenotype) |>
    summarise(nptids = length(unique(patient_id))) |>
    filter(nptids > 1) |>
    ungroup() |>
    group_by(phenotype) |>
    summarize(ngliphs = length(unique(pattern)))
inter_by_cond_tumor_sharing_count 

phenotype,ngliphs
<chr>,<int>
Endemic KS,26735
Epidemic KS,33469


In [137]:
inter_cond_tumor_sharing_count <- kstme_clustered_unknown_table |>
    filter(!str_detect(repertoire_id, "008_\\d+_A")) |>
    group_by(pattern) |>
    summarise(nptids = length(unique(phenotype))) |>
    filter(nptids > 1) |>
    pull(pattern) |>
    unique() |>
    length()
str_c("Number of specificity groups shared by at least two tumors derieved from spatially distinct sites from the epidemic and endemic individuals", inter_cond_tumor_sharing_count, sep = " : ")

[1] "Number of specificity groups shared by at least two tumors derieved from spatially distinct sites from the epidemic and endemic individuals : 35731"

## Calculate Jaccard index between repertoires based on sharing of GLIPH group

In [139]:
kstme_gliph_dist_table <- kstme_gliph_pattern_table |>
    group_by(repertoire_id) |>
    summarize(gliph_groups = list(unique(pattern))) |>
    mutate(gliph_groups = map(gliph_groups, ~sort(as.character(.)))) |>
    ungroup()

In [140]:
kstme_gliph_dist_tibble <- expand_grid(kstme_gliph_dist_table, 
    kstme_gliph_dist_table, .name_repair = "unique") |>
  dplyr::rename(rep_one = repertoire_id...1,
    glist_one = gliph_groups...2,
    rep_two = repertoire_id...3,
    glist_two = gliph_groups...4) |>
  rowwise() |>
  mutate(jaccard = length(intersect(glist_one, glist_two))/length(union(glist_one, glist_two)))  |>
  dplyr::select(rep_one, rep_two, jaccard) |>
  filter(str_detect(rep_one, "008_\\d+") & str_detect(rep_two, "008_\\d+") & 
    !is.na(jaccard) & 
    str_extract(rep_one, "[A-Z]+$") %in% c("A","B", "C", "D") &
    str_extract(rep_two, "[A-Z]+$") %in% c("A","B", "C", "D")) |>
  ungroup() |> 
  mutate(sample_group = if_else(str_extract(rep_one, "008_\\d+") ==
      str_extract(rep_two, "008_\\d+"), "Intra-sample", "Inter-sample"),
    tumor_group = case_when(
      str_extract(rep_one, "[A-Z]+$") == "A" &
        str_extract(rep_two, "[A-Z]+$") == "A" ~ "NAT - NAT",
      str_extract(rep_one, "[A-Z]+$") == "A" &
        str_extract(rep_two, "[A-Z]+$") %in% c("B", "C", "D") ~ "NAT - Tumor",
      str_extract(rep_two, "[A-Z]+$") == "A" &
        str_extract(rep_one, "[A-Z]+$") %in% c("B", "C", "D") ~ "NAT - Tumor",
      str_extract(rep_two, "[A-Z]+$") %in% c("B", "C", "D") &
        str_extract(rep_one, "[A-Z]+$") %in% c("B", "C", "D") ~ "Tumor - Tumor"),
    ks_group = str_c(ptid_dict[str_extract(rep_one, "008_\\d+")],
      ptid_dict[str_extract(rep_two, "008_\\d+")], sep = " - "),
    ks_group = str_replace(ks_group, "Epidemic KS - Endemic KS",
      "Endemic KS - Epidemic KS")) |>
  filter(!(sample_group == "Inter-sample" & tumor_group == "NAT - Tumor") &
    !(rep_one == rep_two))

New names:
• `repertoire_id` -> `repertoire_id...1`
• `gliph_groups` -> `gliph_groups...2`
• `repertoire_id` -> `repertoire_id...3`
• `gliph_groups` -> `gliph_groups...4`


## Figure 4: T-cell repertoires in epidemic and endemic KS tumors are idiosyncratic and contain few TCRs specific for viral/self-antigens but share T-cells clusters with predicted specificity for uncharacterized antigen groups.

In [143]:
### Overlapping TCRs section
endemic_pal <- alluvial_data |>
  filter(str_detect(repertoire_id,  "008_057|008_061")) |>
  topSeqs(top = 500) |>
  cloneTrack() |> 
  group_by(junction_aa) |>
  summarize(reps = str_c(unique(str_extract(repertoire_id, "[ABCD]$")), collapse= ""),
    samples = length(unique(str_extract(repertoire_id, "\\d+_\\d+"))),
    seen = max(seen)) |>
  mutate(pal = case_when(seen == 1 ~ "#f0f0f0",
    samples > 1 & reps != "A" ~ "#e7298a",
    samples > 1 & reps == "A" ~ "#1b9e77",
    samples == 1 & seen > 1 & str_detect(reps, "A") & 
      str_detect(reps, "B|C|D") ~ "#d95f02",
    samples == 1 & seen > 1 & !str_detect(reps, "A") ~ "#7570b3"))
endemic_alluvial <- alluvial_data |>
  filter(str_detect(repertoire_id, "008_057|008_061")) |>
  topSeqs(top = 500) |>
  cloneTrack() |>
  plotTrack(alist = endemic_pal$junction_aa, apal = endemic_pal$pal) +
  scale_x_discrete(limits = c("008_061_A", "008_061_B", "008_061_C", "008_061_D",
    "008_057_D", "008_057_C", "008_057_B", "008_057_A"),
    labels = c("NAT", "Tumor B", "Tumor C", "Tumor D", "Tumor D", "Tumor C",
      "Tumor B", "NAT")) +
  annotate("text", label = "Endemic KS\nStudy Entry", y = 0.95, size = 12, x = 2.5) +
  annotate("text", label = "Endemic KS\nStudy Entry", y = 0.95, size = 12, x = 6.5) +
  geom_vline(xintercept = 4.5, linetype="dotted") + 
  ylim(0,1) +
  theme_classic(base_size = 30) +
  theme(legend.position = "none")


epidemic_pal <- alluvial_data |>
  filter(str_detect(repertoire_id, "008_021|008_048")) |>
  topSeqs(top = 500) |>
  cloneTrack() |> 
  group_by(junction_aa) |>
  summarize(reps = str_c(unique(str_extract(repertoire_id, "[ABCD]$")), collapse= ""),
    samples = length(unique(str_extract(repertoire_id, "\\d+_\\d+"))),
    seen = max(seen)) |>
  mutate(pal = case_when(seen == 1 ~ "#f0f0f0",
    samples > 1 & reps != "A" ~ "#e7298a",
    samples > 1 & reps == "A" ~ "#1b9e77",
    samples == 1 & seen > 1 & str_detect(reps, "A") & 
      str_detect(reps, "B|C|D") ~ "#d95f02",
    samples == 1 & seen > 1 & !str_detect(reps, "A") ~ "#7570b3"))

epidemic_alluvial <- alluvial_data |>
  filter(str_detect(repertoire_id, "008_021|008_048")) |>
  topSeqs(top = 500) |>
  cloneTrack() |>
  plotTrack(alist = epidemic_pal$junction_aa, apal = epidemic_pal$pal) +
  scale_x_discrete(limits = c("008_021_A", "008_021_B", "008_021_C", "008_021_D",
    "008_048_D", "008_048_C", "008_048_B", "008_048_A"),
    labels = c("NAT", "Tumor B", "Tumor C", "Tumor D", "Tumor D", "Tumor C",
      "Tumor B", "NAT")) +
  geom_vline(xintercept = 4.5, linetype="dotted") + 
  annotate("text", label = "Epidemic KS\nStudy Entry", y = 0.95, size = 12, x = 2.5) +
  annotate("text", label = "Epidemic KS\nStudy Entry", y = 0.95, size = 12, x = 6.5) +
  ylim(0,1) +
  theme_classic(base_size = 30) +
  theme(legend.position = "none")

endepi_pal <- alluvial_data |>
  filter(str_detect(repertoire_id,  "008_021|008_057")) |>
  topSeqs(top = 500) |>
  cloneTrack() |> 
  group_by(junction_aa) |>
  summarize(reps = str_c(unique(str_extract(repertoire_id, "[ABCD]$")), collapse= ""),
    samples = length(unique(str_extract(repertoire_id, "\\d+_\\d+"))),
    seen = max(seen)) |>
  mutate(pal = case_when(seen == 1 ~ "#f0f0f0",
    samples > 1 & reps != "A" ~ "#e7298a",
    samples > 1 & reps == "A" ~ "#1b9e77",
    samples == 1 & seen > 1 & str_detect(reps, "A") & 
      str_detect(reps, "B|C|D") ~ "#d95f02",
    samples == 1 & seen > 1 & !str_detect(reps, "A") ~ "#7570b3"))
endepi_alluvial <- alluvial_data |>
  filter(str_detect(repertoire_id, "008_021|008_057")) |>
  topSeqs(top = 500) |>
  cloneTrack() |>
  plotTrack(alist = endepi_pal$junction_aa, apal = endepi_pal$pal) +
  scale_x_discrete(limits = c("008_021_A", "008_021_B", "008_021_C", "008_021_D",
    "008_057_D", "008_057_C", "008_057_B", "008_057_A"),
    labels = c("NAT", "Tumor B", "Tumor C", "Tumor D", "Tumor D", "Tumor C",
      "Tumor B", "NAT")) +
  geom_vline(xintercept = 4.5, linetype="dotted") + 
  annotate("text", label = "Epidemic KS\nStudy Entry", y = 0.95, size = 12, x = 2.5) +
  annotate("text", label = "Endemic KS\nStudy Entry", y = 0.95, size = 12, x = 6.5) +
  ylim(0,1) +
  theme_classic(base_size = 30) +
  theme(legend.position = "none")

endemic_compairr <- compairr_table |>
  mutate(color_group = str_c(sample_group, tumor_group, sep = "\n"),
    overlap = if_else(overlap > 1, 1, overlap)) |>
  filter(ks_group == "Endemic KS - Endemic KS") |>
  ggboxplot(x = "color_group", y = "overlap", fill = "color_group",
    order = c("Inter-subject\nNAT - NAT", "Intra-subject\nNAT - Tumor",
      "Intra-subject\nTumor - Tumor", "Inter-subject\nTumor - Tumor")) |>
  ggpar(xlab = FALSE, ylab = "Morisita-Horn index", 
    ggtheme = theme_classic(base_size = 30), legend = "none",
    legend.title = "Group", palette = c("Inter-subject\nNAT - NAT" = "#1b9e77", 
      "Intra-subject\nNAT - Tumor" = "#d95f02",
      "Intra-subject\nTumor - Tumor" = "#7570b3", 
      "Inter-subject\nTumor - Tumor" = "#e7298a")) +
  stat_compare_means(comparisons = list(c("Intra-subject\nNAT - Tumor",
      "Intra-subject\nTumor - Tumor")), size = 6,  label = "p.signif") +
  scale_y_continuous(expand = expansion(mult = c(0.05, 0.25)))

epidemic_compairr <- compairr_table |>
  mutate(color_group = str_c(sample_group, tumor_group, sep = "\n"),
    overlap = if_else(overlap > 1, 1, overlap)) |>
  filter(ks_group == "Epidemic KS - Epidemic KS") |>
  ggboxplot(x = "color_group", y = "overlap", fill = "color_group",
    order = c("Inter-subject\nNAT - NAT", "Intra-subject\nNAT - Tumor",
      "Intra-subject\nTumor - Tumor", "Inter-subject\nTumor - Tumor")) |>
  ggpar(xlab = FALSE, ylab = "Morisita-Horn index", 
    ggtheme = theme_classic(base_size = 30), legend = "right",
    legend.title = "Group", palette = c("Inter-subject\nNAT - NAT" = "#1b9e77", 
      "Intra-subject\nNAT - Tumor" = "#d95f02",
      "Intra-subject\nTumor - Tumor" = "#7570b3", 
      "Inter-subject\nTumor - Tumor" = "#e7298a")) +
  stat_compare_means(comparisons = list(c("Intra-subject\nNAT - Tumor",
      "Intra-subject\nTumor - Tumor")), size = 6,  label = "p.signif") +
  scale_y_continuous(expand = expansion(mult = c(0.05, 0.25)))

endemic_epidemic_compairr <- compairr_table |>
  mutate(color_group = str_c(sample_group, tumor_group, sep = "\n"),
    overlap = if_else(overlap > 1, 1, overlap)) |>
  filter(ks_group == "Endemic KS - Epidemic KS") |>
  ggboxplot(x = "color_group", y = "overlap", fill = "color_group",
    order = c("Inter-subject\nNAT - NAT", "Inter-subject\nTumor - Tumor")) |>
  ggpar(xlab = FALSE, ylab = "Morisita-Horn index", ylim = c(0,1),
    ggtheme = theme_classic(base_size = 30),
    legend = "none", palette = c("Inter-subject\nNAT - NAT" = "#1b9e77", 
      "Intra-subject\nNAT - Tumor" = "#d95f02",
      "Intra-subject\nTumor - Tumor" = "#7570b3", 
      "Inter-subject\nTumor - Tumor" = "#e7298a"), size = 6, label = "p.signif")
### Public TCRs section
public_tcrs_plot <- path_annotated_nprod_table |>
  mutate(frequency = frequency * 100) |>
  filter(cohort != "ARKS") |>
  ggboxplot(x = "pathology", y = "frequency", color = "cohort", 
    add = "jitter", order = c("CMV", "DENV1", "DENV3/4", "EBV", "HCV", "HIV-1",
       "HSV-2",
      "InfluenzaA", "M.tuberculosis", "SARS-CoV-2", "Other", "Multi-pathogen",
      "Unknown")) |>
  ggpar(xlab = FALSE, ylab = "Cummulative frequency", 
    legend.title = "Cohort", 
    format.scale = T, legend = "right",
    ggtheme = theme_classic(base_size = 30),
    palette = c("Epidemic KS - NAT" = "#fb8072", 
  "Epidemic KS - Tumor" = "#e41a1c", "Endemic KS - NAT" = "#80b1d3",
  "Endemic KS - Tumor" = "#377eb8")) +
  geom_hline(yintercept = 1, linetype = 2) + 
  geom_hline(yintercept = 25, linetype = 2) + 
  geom_hline(yintercept = 75, linetype = 2) + 
  geom_hline(yintercept = 90, linetype = 2) + 
  scale_y_log10(breaks = scales::trans_breaks("log10", function(x) 10^x),
    labels = scales::trans_format("log10", scales::math_format(10^.x))) +
  annotation_logticks(sides = "l", short = unit(2,"mm"),
    mid = unit(4,"mm"),long = unit(8,"mm"), size = 1) +
  theme_classic(base_size = 30) +
  theme(axis.text = element_text(face = "bold"),
    axis.title.x = element_blank())

public_gliph_plot <- path_annotated_gliph_nprod_table |>
  mutate(frequency = frequency * 100) |>
  ggboxplot(x = "pathology", y = "frequency", color = "cohort", 
    add = "jitter", order = c("CMV", "EBV", "HCV", "HIV-1",
       "HSV-2", "InfluenzaA", "M.tuberculosis", "SARS-CoV-2", "Other", 
       "Multi-pathogen",   "Clustered\nUnknown", "Unclustered\nUnknown")) |>
  ggpar(xlab = FALSE, ylab = "Cummulative frequency", 
    legend.title = "Cohort", 
    format.scale = T, legend = "right",
    ggtheme = theme_classic(base_size = 30),
    palette = c("Epidemic KS - NAT" = "#fb8072", 
  "Epidemic KS - Tumor" = "#e41a1c", "Endemic KS - NAT" = "#80b1d3",
  "Endemic KS - Tumor" = "#377eb8")) +
  geom_hline(yintercept = 1, linetype = 2) +
  geom_hline(yintercept = 25, linetype = 2) +
  geom_hline(yintercept = 75, linetype = 2) +  
  geom_hline(yintercept = 90, linetype = 2) +  
  scale_y_log10(breaks = scales::trans_breaks("log10", function(x) 10^x),
    labels = scales::trans_format("log10", scales::math_format(10^.x))) +
  annotation_logticks(sides = "l", short = unit(2,"mm"),
    mid = unit(4,"mm"),long = unit(8,"mm"), size = 1) + 
  theme_classic(base_size = 30) +
  theme(axis.text = element_text(face = "bold"),
    axis.title.x = element_blank())

### Overlapping clustered unknown section
endemic_gliph_compairr <- kstme_gliph_dist_tibble |>
  mutate(color_group = str_c(str_replace(sample_group, "sample", "subject"), tumor_group, sep = "\n"),
    jaccard = if_else(jaccard > 1, 1, jaccard)) |>
  filter(ks_group == "Endemic KS - Endemic KS") |>
  ggboxplot(x = "color_group", y = "jaccard", fill = "color_group",
    order = c("Inter-subject\nNAT - NAT", "Intra-subject\nNAT - Tumor",
      "Intra-subject\nTumor - Tumor", "Inter-subject\nTumor - Tumor")) |>
  ggpar(xlab = FALSE, ylab = "Jaccard index",  #title = "Endemic KS - Endemic KS",
    ggtheme = theme_classic(base_size = 30), legend = "none",
    legend.title = "Group", palette = c("Inter-subject\nNAT - NAT" = "#1b9e77", 
      "Intra-subject\nNAT - Tumor" = "#d95f02",
      "Intra-subject\nTumor - Tumor" = "#7570b3", 
      "Inter-subject\nTumor - Tumor" = "#e7298a"),
    ylim = c(0,1)) +  stat_compare_means(comparisons = list(c("Intra-subject\nNAT - Tumor",
      "Intra-subject\nTumor - Tumor")), size = 6)

epidemic_gliph_compairr <- kstme_gliph_dist_tibble |>
  mutate(color_group = str_c(str_replace(sample_group, "sample", "subject"), tumor_group, sep = "\n"),
    jaccard = if_else(jaccard > 1, 1, jaccard)) |>
  filter(ks_group == "Epidemic KS - Epidemic KS") |>
  ggboxplot(x = "color_group", y = "jaccard", fill = "color_group",
    order = c("Inter-subject\nNAT - NAT", "Intra-subject\nNAT - Tumor",
      "Intra-subject\nTumor - Tumor", "Inter-subject\nTumor - Tumor")) |>
  ggpar(xlab = FALSE, ylab = "Jaccard index",  #title = "Epidemic KS - Epidemic KS",
    ggtheme = theme_classic(base_size = 30), legend = "right",
    legend.title = "Group", palette = c("Inter-subject\nNAT - NAT" = "#1b9e77", 
      "Intra-subject\nNAT - Tumor" = "#d95f02",
      "Intra-subject\nTumor - Tumor" = "#7570b3", 
      "Inter-subject\nTumor - Tumor" = "#e7298a"),
    ylim = c(0,1)) +
  stat_compare_means(comparisons = list(c("Intra-subject\nNAT - Tumor",
      "Intra-subject\nTumor - Tumor")), size = 6) 

endemic_epidemic_gliph_compairr <- kstme_gliph_dist_tibble |>
  mutate(color_group = str_c(str_replace(sample_group, "sample", "subject"), tumor_group, sep = "\n"),
    jaccard = if_else(jaccard > 1, 1, jaccard)) |>
  filter(ks_group == "Endemic KS - Epidemic KS") |>
  ggboxplot(x = "color_group", y = "jaccard", fill = "color_group",
    order = c("Inter-subject\nNAT - NAT", "Inter-subject\nTumor - Tumor")) |>
  ggpar(xlab = FALSE, ylab = "Jaccard index", ylim = c(0,1),
    ggtheme = theme_classic(base_size = 30), #title = "Endemic KS - Epidemic KS",
    legend = "none", palette = c("Inter-sample\nNAT - NAT" = "#1b9e77", 
      "Intra-subject\nNAT - Tumor" = "#d95f02",
      "Intra-subject\nTumor - Tumor" = "#7570b3", 
      "Inter-subject\nTumor - Tumor" = "#e7298a")) +
  stat_compare_means(comparisons = list(c("Intet-subject\nNAT - NAT",
      "Intra-subject\nTumor - Tumor")), size = 6) 


figure_four_layout <- "
AAAABBBBCCCC
AAAABBBBCCCC
AAAABBBBCCCC
DDDDEEEEFFFF
DDDDEEEEFFFF
GGGGGGGGGGGG
GGGGGGGGGGGG
HHHHHHHHHHHH
HHHHHHHHHHHH
IIIIJJJJKKKK
IIIIJJJJKKKK
"

figure_four <- endemic_alluvial + 
  epidemic_alluvial + 
  endepi_alluvial +
  endemic_compairr +
  epidemic_compairr +
  endemic_epidemic_compairr + 
  public_tcrs_plot + 
  public_gliph_plot +
  endemic_gliph_compairr +
  epidemic_gliph_compairr +
  endemic_epidemic_gliph_compairr + 
  plot_layout(design = figure_four_layout, guides = "collect") +
  plot_annotation(tag_levels = "A") &
  theme(plot.tag = element_text(size = 42,face = "bold"))

ggsave(str_c(figures_path, "pdf", "Figure_four.pdf", sep = "/"), figure_four, 
  width = 48, height = 42, device = cairo_pdf, family = "Arial Unicode MS")
ggsave(str_c(figures_path, "svg", "Figure_four.svg", sep = "/"), figure_four, 
  width = 48, height = 42)
ggsave(str_c(figures_path, "png", "Figure_four.png", sep = "/"), figure_four, 
  width = 48, height = 42)


Warning message in wilcox.test.default(c(0.1583104587, 0.162004413, 0.1557680306, :
“cannot compute exact p-value with ties”
Warning message:
“Computation failed in `stat_signif()`
Caused by error in `if (scales$x$map(comp[1]) == data$group[1] | manual) ...`:
! missing value where TRUE/FALSE needed”
Warning message in wilcox.test.default(c(0.1583104587, 0.162004413, 0.1557680306, :
“cannot compute exact p-value with ties”
Warning message:
“Computation failed in `stat_signif()`
Caused by error in `if (scales$x$map(comp[1]) == data$group[1] | manual) ...`:
! missing value where TRUE/FALSE needed”
Warning message in wilcox.test.default(c(0.1583104587, 0.162004413, 0.1557680306, :
“cannot compute exact p-value with ties”
Warning message:
“Computation failed in `stat_signif()`
Caused by error in `if (scales$x$map(comp[1]) == data$group[1] | manual) ...`:
! missing value where TRUE/FALSE needed”


## Supplementary Figure 3: Frequency of T-cells in GLIPH2-defined clusters with unknown antigenic specificity in tumors and NAT samples from individuals with epidemic and endemic KS.

In [144]:
supplementary_figure_three <- kstme_clustered_unknown_table |>
    mutate(duplicate_frequency = duplicate_frequency * 100,
        group = if_else(str_detect(repertoire_id, "008_\\d+"), 
          str_c(phenotype, tissue_type, sep = " - "), phenotype),
        group = factor(group, levels = c("Epidemic KS - NAT", 
          "Epidemic KS - Tumor", "Endemic KS - NAT",
          "Endemic KS - Tumor", "BL- Ghana", "BL - Uganda"))) |>
    ggboxplot(x = "group", y = "duplicate_frequency", color = "group") |>
    ggpar(xlab = FALSE, ylab = "Cummulative frequency", 
    legend.title = "Cohort", 
    format.scale = T, legend = "right",
    ggtheme = theme_classic(base_size = 32),
    palette = c("Epidemic KS - NAT" = "#fb8072", 
  "Epidemic KS - Tumor" = "#e41a1c", "Endemic KS - NAT" = "#80b1d3",
  "Endemic KS - Tumor" = "#377eb8")) +
  geom_hline(yintercept = 1, linetype = 2) +
  geom_hline(yintercept = 10, linetype = 2) +
  geom_hline(yintercept = 75, linetype = 2) +  
  scale_y_log10(breaks = scales::trans_breaks("log10", function(x) 10^x),
    labels = scales::trans_format("log10", scales::math_format(10^.x))) +
  annotation_logticks(sides = "l", short = unit(2,"mm"),
    mid = unit(4,"mm"),long = unit(8,"mm"), size = 1) +
  theme(axis.text =  element_text(face = "bold"))
ggsave(str_c(figures_path, "pdf", "Supplementary_figure_three.pdf", sep = "/"), supplementary_figure_three, 
  width = 24, height = 10)
ggsave(str_c(figures_path, "svg", "Supplementary_figure_three.svg", sep = "/"), supplementary_figure_three, 
  width = 24, height = 10)
ggsave(str_c(figures_path, "png", "Supplementary_figure_thre.png", sep = "/"), supplementary_figure_three, 
  width = 24, height = 10)

## Figure 5: Synchronously acquired, non-contiguous tumors from endemic KS show a greater degree of intra-tumor TCR sharing than epidemic KS, and inter-tumor sharing of TCR sequences is low both within and across epidemic and endemic KS. 

In [149]:
endemic_upset_input_table <- alluvial_data |>
  filter(str_detect(repertoire_id,  "008_057|008_061"))
endemic_upset_input_table

repertoire_id,junction_aa,v_call,d_call,j_call,v_family,d_family,j_family,reading_frame,duplicate_count,duplicate_frequency
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
008_057_A,CAGGGRGAEDTQYF,TRBV7-8,TRBD2-1,TRBJ2-3,TRBV7,TRBD2,TRBJ2,in-frame,1,0.0007429421
008_057_A,CAGRREETQYF,TRBV2-1,TRBD1-1,TRBJ2-5,TRBV2,TRBD1,TRBJ2,in-frame,1,0.0007429421
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
008_061_D,WPAAATIPFHSDNEQFF,NA,TRBD2-1,TRBJ2-1,NA,TRBD2,TRBJ2,in-frame,1,6.928566e-05


In [152]:
# UpSetR plots for endemic KS comparisons

endemic_upset_input_tibble <- alluvial_data |>
    filter(str_detect(repertoire_id,  "008_057|008_061"))  |>
    left_join(study_metadata, by = "repertoire_id") |>
    dplyr::select(repertoire_id, junction_aa, tissue_type, phenotype) |>
    mutate(present = 1,
        row_id = str_c(repertoire_id, tissue_type, phenotype, sep = " ")) |>
    pivot_wider(id_cols = junction_aa, names_from = repertoire_id, values_from = present, values_fill = 0) |>
    dplyr::select(junction_aa, "008_057_A", "008_057_B", "008_057_C", "008_057_D", "008_061_D",
        "008_061_C", "008_061_B", "008_061_A")

endemic_upset_rownames <- endemic_upset_input_tibble |>
    pull(junction_aa)
endemic_upset_input_table <- endemic_upset_input_tibble |>
    dplyr::select(-junction_aa) |>
    as.data.frame() 
rownames(endemic_upset_input_table) <- endemic_upset_rownames

endemic_upset <- upset(endemic_upset_input_table, 
    intersect = c("008_057_A", "008_057_B", "008_057_C", "008_057_D", "008_061_D", "008_061_C", "008_061_B", "008_061_A"),
    queries = list(
    upset_query(set = '008_061_A', fill = '#80b1d3'),
    upset_query(set = '008_061_B', fill = '#377eb8'),
    upset_query(set = '008_061_C', fill = '#377eb8'),
    upset_query(set = '008_061_D', fill = '#377eb8'),
    upset_query(set = '008_057_A', fill = '#80b1d3'),
    upset_query(set = '008_057_B', fill = '#377eb8'),
    upset_query(set = '008_057_C', fill = '#377eb8'),
    upset_query(set = '008_057_D', fill = '#377eb8'),
    upset_query(intersect = c('008_061_A'), color = '#80b1d3', fill = '#80b1d3', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_057_A'), color = '#80b1d3', fill = '#80b1d3', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_061_B'), color = '#377eb8', fill = '#377eb8', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_061_C'), color = '#377eb8', fill = '#377eb8', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_061_D'), color = '#377eb8', fill = '#377eb8', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_057_B'), color = '#377eb8', fill = '#377eb8', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_057_C'), color = '#377eb8', fill = '#377eb8', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_057_D'), color = '#377eb8', fill = '#377eb8', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_061_B','008_061_D'), color = '#7570b3', fill = '#7570b3', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_057_B','008_057_D'), color = '#7570b3', fill = '#7570b3', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_057_B','008_057_C', '008_057_D'), color = '#7570b3', fill = '#7570b3', only_components=c('intersections_matrix', 'Intersection size')),    
    upset_query(intersect = c('008_061_B','008_061_C', '008_061_D'), color = '#7570b3', fill = '#7570b3', only_components=c('intersections_matrix', 'Intersection size')),    
    upset_query(intersect = c('008_057_B','008_057_C'), color = '#7570b3', fill = '#7570b3', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_057_C','008_057_D'), color = '#7570b3', fill = '#7570b3', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_061_B','008_061_C'), color = '#7570b3', fill = '#7570b3', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_061_C','008_061_D'), color = '#7570b3', fill = '#7570b3', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_057_A','008_057_B', '008_057_C', '008_057_D'), color = '#d95f02', fill = '#d95f02', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_061_A','008_061_B', '008_061_D'), color = '#d95f02', fill = '#d95f02', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_061_A','008_061_B'), color = '#d95f02', fill = '#d95f02', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_061_A','008_061_B', '008_061_C', '008_061_D'), color = '#d95f02', fill = '#d95f02', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_061_A','008_061_D'), color = '#d95f02', fill = '#d95f02', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_057_A','008_057_D'), color = '#d95f02', fill = '#d95f02', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_057_A','008_057_B'), color = '#d95f02', fill = '#d95f02', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_057_A','008_057_B', '008_057_D'), color = '#d95f02', fill = '#d95f02', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_057_A','008_057_C'), color = '#d95f02', fill = '#d95f02', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_057_A','008_057_C', '008_057_D'), color = '#d95f02', fill = '#d95f02', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_061_A','008_061_B', '008_061_C'), color = '#d95f02', fill = '#d95f02', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_061_A','008_061_C'), color = '#d95f02', fill = '#d95f02', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_061_B','008_057_B'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_057_A','008_057_B', '008_057_C'), color = '#d95f02', fill = '#d95f02', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_061_D','008_057_D'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_061_B','008_057_C'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_061_B','008_057_D'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_061_A','008_061_C', '008_061_D'), color = '#d95f02', fill = '#d95f02', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_061_D','008_057_B'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_061_B','008_057_D', '008_057_C', '008_057_B'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_061_A','008_057_D'), color = '#606060', fill = '#606060', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_061_D','008_057_C'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_061_D','008_057_D', '008_057_B'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_061_B','008_057_D', '008_057_B'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_061_B','008_061_C', '008_057_B'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_061_B','008_061_C', '008_057_C'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_061_C','008_057_C'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_061_B','008_061_C', '008_061_D', '008_057_B'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_061_B','008_061_D', '008_057_B'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_061_C','008_057_B'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_061_A','008_057_B'), color = '#606060', fill = '#606060', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_061_A','008_057_D','008_057_C','008_057_B'), color = '#606060', fill = '#606060', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_061_B','008_061_D', '008_057_D'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_061_B','008_061_C', '008_061_D', '008_057_C'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_061_C','008_061_D', '008_057_C'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),    
    upset_query(intersect = c('008_057_C','008_061_A'), color = '#606060', fill = '#606060', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_061_B','008_057_D','008_057_C'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_061_A','008_061_B', '008_057_B'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_061_C','008_061_D', '008_057_D', '008_057_B'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_061_A','008_061_D', '008_057_D', '008_057_B'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_061_C','008_057_D', '008_057_B'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_061_C','008_061_D', '008_057_C', '008_057_B'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_061_A','008_061_B', '008_061_D', '008_057_C', '008_057_B'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_061_B','008_057_C', '008_057_B'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_061_D','008_057_D', '008_057_C', '008_057_B'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_061_A','008_061_D','008_057_A'), color = '#606060', fill = '#606060', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_061_B','008_057_A'), color = '#606060', fill = '#606060', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_061_A','008_057_A'), color = '#1b9e77', fill = '#1b9e77', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_061_B','008_057_D','008_057_A'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_061_B','008_061_C','008_061_D', '008_057_C', '008_057_A'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_061_D','008_057_B','008_057_A'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_061_A','008_057_B','008_057_A'), color = '#606060', fill = '#606060', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_061_A','008_057_D','008_057_B','008_057_A'), color = '#606060', fill = '#606060', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_061_D','008_057_D','008_057_C','008_057_B','008_057_A'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_061_C','008_057_D','008_057_C','008_057_B','008_057_A'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
    upset_query(intersect = c('008_061_A','008_057_D','008_057_C','008_057_B','008_057_A'), color = '#606060', fill = '#606060', only_components=c('intersections_matrix', 'Intersection size'))),
    intersections = list('008_061_B', '008_057_B','008_057_D', '008_061_D',
        '008_057_C', '008_061_A', '008_061_C', '008_057_A', 
        c('008_061_B','008_061_D'),
        c('008_061_B','008_061_C','008_061_D'),
        c('008_061_B','008_061_C'),
        c('008_061_D','008_061_C'),
        c('008_057_D','008_057_B'),
        c('008_057_D','008_057_C','008_057_B'), 
        c('008_057_C','008_057_B'),
        c('008_057_D','008_057_C'),
        c('008_061_A','008_061_B','008_061_D'),
        c('008_061_A','008_061_B'),
        c('008_061_A','008_061_B','008_061_C','008_061_D'),
        c('008_061_A','008_061_D'),
        c('008_061_A','008_061_B','008_061_C'),
        c('008_061_A','008_061_C'),
        c('008_061_A','008_061_C','008_061_D'),
        c('008_057_D','008_057_C','008_057_B','008_057_A'),
        c('008_057_D','008_057_A'),
        c('008_057_B','008_057_A'),
        c('008_057_D','008_057_B','008_057_A'),
        c('008_057_C','008_057_A'),
        c('008_057_D','008_057_C','008_057_A'),
        c('008_057_C','008_057_B','008_057_A'),
        c('008_057_B','008_061_B'),
        c('008_057_D','008_061_D'),
        c('008_061_B','008_057_C'),
        c('008_061_B','008_057_D'),
        c('008_061_D','008_057_B'),
        c('008_061_B','008_057_D','008_057_C','008_057_B'),
        c('008_061_D','008_057_C'),
        c('008_061_D','008_057_D','008_057_B'),
        c('008_061_B','008_057_D','008_057_B'),
        c('008_061_B','008_061_C','008_057_B'),
        c('008_061_B','008_061_C','008_057_C'),
        c('008_061_C','008_057_C'),
        c('008_061_B','008_061_C','008_061_D','008_057_B'),
        c('008_061_B','008_061_D','008_057_B'),
        c('008_061_C','008_057_B'),
        c('008_061_B','008_061_D','008_057_D'),
        c('008_061_B','008_061_C','008_061_D','008_057_C'),
        c('008_061_C','008_061_D','008_057_C'),
        c('008_061_B','008_057_D','008_057_C'),
        c('008_061_A','008_061_B','008_057_B'),
        c('008_061_C','008_061_D','008_057_D','008_057_B'),
        c('008_061_A','008_061_D','008_057_D','008_057_B'),
        c('008_061_C','008_057_D','008_057_B'),
        c('008_061_C','008_061_D','008_057_C','008_057_B'),
        c('008_061_A','008_061_B','008_061_D','008_057_C','008_057_B'),
        c('008_061_B','008_057_C','008_057_B'),
        c('008_061_D','008_057_D','008_057_C','008_057_B'),
        c('008_061_B','008_057_D','008_057_A'),
        c('008_061_B','008_061_C','008_061_D','008_057_C','008_057_A'),
        c('008_061_D','008_057_B','008_057_A'),
        c('008_061_D','008_057_D','008_057_C','008_057_B','008_057_A'),
        c('008_061_C','008_057_D','008_057_C','008_057_B','008_057_A'),
        c('008_061_A','008_057_A'),
        c('008_061_A','008_057_D'),    
        c('008_061_A','008_057_B'),
        c('008_061_A','008_057_D','008_057_C','008_057_B'),
        c('008_061_A','008_057_C'),
        c('008_061_A','008_061_D','008_057_A'),
        c('008_061_B','008_057_A'),
        c('008_061_A','008_057_B','008_057_A'),
        c('008_061_A','008_057_D','008_057_B','008_057_A'),
        c('008_061_A','008_057_D','008_057_C','008_057_B','008_057_A')),
    sort_sets=FALSE,
    sort_intersections=FALSE,
    width_ratio=0.1,
    name = "Overlap between TCR repertoires",
    base_annotations = list(
            'Intersection size'=intersection_size(
                text = list(size = 7, angle = 90, hjust = 0, vjust = 0.5)
            ) + ylab('Number of overlapping TCRs') 
            + ggtitle("Endemic KS - Endemic KS")
            + theme_classic() 
            + labs(tag = "A")
            + theme(axis.title.x = element_blank(),
                plot.title = element_text(size = 30, face = "bold", family = "NimbusSan", hjust = 0.5),
                axis.text.x = element_blank(),
                axis.ticks.x = element_blank(),
                axis.title.y = element_text(size = 24, face = "bold", family = "NimbusSan"),
                axis.text.y = element_text(size = 22, face = "bold", family = "NimbusSan"),
                plot.tag = element_text(size = 28, face = "bold", family = "NimbusSan"))),
    set_sizes=(upset_set_size()+ ylab('Number of\nunique TCRs') 
            + theme_void() 
            + theme(axis.title.y = element_blank(),
                axis.text.y = element_blank(),
                axis.ticks.y = element_blank(),
                axis.title.x = element_blank(),
                axis.text.x = element_text(size = 20, face = "bold", family = "NimbusSan", angle = 90, vjust = 0.5),
                axis.ticks.x = element_line(linewidth = 2, colour = 'black' ))
    )) +
    theme(axis.title.x = element_blank(),
        axis.text.y = element_text(size = 24, face = "bold", family = "NimbusSan"))


In [153]:
# UpSetR plots for epidemic KS comparisons
epidemic_upset_input_tibble <- alluvial_data |>
    filter(str_detect(repertoire_id, "008_021|008_048"))  |>
    left_join(study_metadata, by = "repertoire_id") |>
    dplyr::select(repertoire_id, junction_aa, tissue_type, phenotype) |>
    mutate(present = 1,
        row_id = str_c(repertoire_id, tissue_type, phenotype, sep = " ")) |>
    pivot_wider(id_cols = junction_aa, names_from = repertoire_id, values_from = present, values_fill = 0)

epidemic_upset_rownames <- epidemic_upset_input_tibble |>
    pull(junction_aa)
epidemic_upset_input_table <- epidemic_upset_input_tibble |>
    dplyr::select(-junction_aa) |>
    as.data.frame() 
rownames(epidemic_upset_input_table) <- epidemic_upset_rownames

epidemic_upset <- upset(epidemic_upset_input_tibble, 
    intersect = c("008_021_A", "008_021_B", "008_021_C", "008_021_D", "008_048_D", "008_048_C", "008_048_B", "008_048_A"),
    queries = list(
        upset_query(set = '008_021_A', fill = '#fb8072'),
        upset_query(set = '008_021_B', fill = '#e41a1c'),
        upset_query(set = '008_021_C', fill = '#e41a1c'),
        upset_query(set = '008_021_D', fill = '#e41a1c'),
        upset_query(set = '008_048_A', fill = '#fb8072'),
        upset_query(set = '008_048_B', fill = '#e41a1c'),
        upset_query(set = '008_048_C', fill = '#e41a1c'),
        upset_query(set = '008_048_D', fill = '#e41a1c'),
        upset_query(intersect = c('008_048_A','008_048_B','008_048_C','008_048_D'), color = '#d95f02', fill = '#d95f02', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_048_B','008_048_C','008_048_D'), color = '#7570b3', fill = '#7570b3', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_048_A','008_048_C','008_048_D'), color = '#d95f02', fill = '#d95f02', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_048_C','008_048_D'), color = '#7570b3', fill = '#7570b3', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_048_A','008_048_B','008_048_D'), color = '#d95f02', fill = '#d95f02', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_048_B','008_048_D'), color = '#7570b3', fill = '#7570b3', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_048_A','008_048_D'), color = '#d95f02', fill = '#d95f02', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_048_D'), color = '#e41a1c', fill = '#e41a1c', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_048_A','008_048_B','008_048_C'), color = '#d95f02', fill = '#d95f02', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_048_B','008_048_C'), color = '#7570b3', fill = '#7570b3', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_048_A','008_048_C'), color = '#d95f02', fill = '#d95f02', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_048_C'), color = '#e41a1c', fill = '#e41a1c', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_048_A','008_048_B'), color = '#d95f02', fill = '#d95f02', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_048_B'), color = '#e41a1c', fill = '#e41a1c', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_048_A'), color = '#fb8072', fill = '#fb8072', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_048_B','008_048_D','008_021_D'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_048_D','008_021_D'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_048_B','008_021_D'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_021_D'), color = '#e41a1c', fill = '#e41a1c', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_048_A','008_048_B','008_048_C','008_048_D','008_021_C'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_048_B','008_048_C','008_048_D','008_021_C'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_048_C','008_048_D','008_021_C'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_048_B','008_048_D','008_021_C'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_048_C','008_021_C'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_048_B','008_021_C'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_048_A','008_048_B','008_048_C','008_048_D','008_021_D','008_021_C'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_048_B','008_048_C','008_048_D','008_021_D','008_021_C'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_048_B','008_048_D','008_021_D','008_021_C'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_048_D','008_021_D','008_021_C'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_021_D','008_021_C'), color = '#7570b3', fill = '#7570b3', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_021_C'), color = '#e41a1c', fill = '#e41a1c', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_048_A','008_048_B','008_048_C','008_048_D','008_021_B'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_048_B','008_048_D','008_021_B'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_048_D','008_021_B'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_048_B','008_021_B'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_048_A','008_021_B'), color = '#606060', fill = '#606060', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_048_D','008_021_D','008_021_B'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_021_D','008_021_B'), color = '#7570b3', fill = '#7570b3', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_048_D','008_021_C','008_021_B'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_048_C','008_021_C','008_021_B'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_021_D','008_021_C','008_021_B'), color = '#7570b3', fill = '#7570b3', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_021_C','008_021_B'), color = '#7570b3', fill = '#7570b3', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_021_B'), color = '#e41a1c', fill = '#e41a1c', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_048_D','008_021_A'), color = '#606060', fill = '#606060', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_048_B','008_021_A'), color = '#606060', fill = '#606060', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_048_D','008_021_D','008_021_A'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_021_D','008_021_A'), color = '#d95f02', fill = '#d95f02', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_021_D','008_021_C','008_021_A'), color = '#d95f02', fill = '#d95f02', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_021_C','008_021_A'), color = '#d95f02', fill = '#d95f02', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_021_D','008_021_B','008_021_A'), color = '#d95f02', fill = '#d95f02', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_048_D','008_021_C','008_021_B','008_021_A'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_048_A','008_048_C','008_048_D','008_021_D','008_021_C','008_021_B','008_021_A'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_048_D','008_021_D','008_021_C','008_021_B','008_021_A'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_048_B','008_021_D','008_021_C','008_021_B','008_021_A'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_048_A','008_021_D','008_021_C','008_021_B','008_021_A'), color = '#606060', fill = '#606060', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_021_D','008_021_C','008_021_B','008_021_A'), color = '#d95f02', fill = '#d95f02', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_021_C','008_021_B','008_021_A'), color = '#d95f02', fill = '#d95f02', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_021_B','008_021_A'), color = '#d95f02', fill = '#d95f02', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_021_A'), color = '#fb8072', fill = '#fb8072', only_components=c('intersections_matrix', 'Intersection size'))
    ),
    intersections = list(
        '008_048_B','008_048_D','008_021_C','008_021_D',
        '008_048_A','008_021_B','008_048_C','008_021_A',
        c('008_048_B','008_048_D'),
        c('008_048_B','008_048_C','008_048_D'),
        c('008_048_C','008_048_D'),
        c('008_048_B','008_048_C'),
        c('008_021_D','008_021_C'),
        c('008_021_D','008_021_C','008_021_B'),
        c('008_021_C','008_021_B'),
        c('008_021_D','008_021_B'),
        c('008_048_A','008_048_B','008_048_C','008_048_D'),
        c('008_048_A','008_048_B','008_048_D'),
        c('008_048_A','008_048_B'),
        c('008_048_A','008_048_D'),
        c('008_048_A','008_048_B','008_048_C'),
        c('008_048_A','008_048_C','008_048_D'),
        c('008_048_A','008_048_C'),
        c('008_021_D','008_021_C','008_021_B','008_021_A'),
        c('008_021_D','008_021_C','008_021_A'),
        c('008_021_B','008_021_A'),
        c('008_021_C','008_021_A'),
        c('008_021_C','008_021_B','008_021_A'),
        c('008_021_D','008_021_B','008_021_A'),
        c('008_021_D','008_021_A'),
        c('008_048_B','008_021_C'),
        c('008_048_D','008_021_D'),
        c('008_048_B','008_021_D','008_021_C','008_021_B','008_021_A'),
        c('008_048_D','008_021_B'),
        c('008_048_A','008_048_B','008_048_C','008_048_D','008_021_D','008_021_C'),
        c('008_048_B','008_048_D','008_021_C'),
        c('008_048_B','008_048_D','008_021_B'),
        c('008_048_B','008_021_B'),
        c('008_048_B','008_048_D','008_021_D'),
        c('008_048_B','008_021_D'),
        c('008_048_A','008_048_B','008_048_C','008_048_D','008_021_C'),
        c('008_048_B','008_048_C','008_048_D','008_021_C'),
        c('008_048_C','008_048_D','008_021_C'),
        c('008_048_C','008_021_C'),
        c('008_048_B','008_048_C','008_048_D','008_021_D','008_021_C'),
        c('008_048_B','008_048_D','008_021_D','008_021_C'),
        c('008_048_D','008_021_D','008_021_C'),
        c('008_048_A','008_048_B','008_048_C','008_048_D','008_021_B'),
        c('008_048_D','008_021_D','008_021_B'),
        c('008_048_D','008_021_C','008_021_B'),
        c('008_048_C','008_021_C','008_021_B'),
        c('008_048_D','008_021_D','008_021_A'),
        c('008_048_D','008_021_C','008_021_B','008_021_A'),
        c('008_048_A','008_048_C','008_048_D','008_021_D','008_021_C','008_021_B','008_021_A'),
        c('008_048_D','008_021_D','008_021_C','008_021_B','008_021_A'),
        c('008_048_A','008_021_D','008_021_C','008_021_B','008_021_A'),
        c('008_048_A','008_021_B'),
        c('008_048_D','008_021_A'),
        c('008_048_B','008_021_A')),
    sort_sets=FALSE,
    sort_intersections=FALSE,
    width_ratio=0.1,
    name = "Overlap between TCR repertoires",
    base_annotations = list(
            'Intersection size'=intersection_size(
                text = list(size = 7, angle = 90, hjust = 0, vjust = 0.5)
            ) + ylab('Number of overlapping TCRs') 
            + ggtitle("Epidemic KS - Epidemic KS")
            + theme_classic() 
            + labs(tag = "B")
            + theme(axis.title.x = element_blank(),
                plot.title = element_text(size = 30, face = "bold", family = "NimbusSan", hjust = 0.5),
                axis.text.x = element_blank(),
                axis.ticks.x = element_blank(),
                axis.title.y = element_text(size = 24, face = "bold", family = "NimbusSan"),
                axis.text.y = element_text(size = 22, face = "bold", family = "NimbusSan"),
                plot.tag = element_text(size = 28, face = "bold", family = "NimbusSan"))),
    set_sizes=(upset_set_size()+ ylab('Number of\nunique TCRs') 
            + theme_void() 
            + theme(axis.title.y = element_blank(),
                axis.text.y = element_blank(),
                axis.ticks.y = element_blank(),
                axis.title.x = element_blank(),
                axis.text.x = element_text(size = 20, face = "bold", family = "NimbusSan", angle = 90, vjust = 0.5),
                axis.ticks.x = element_line(linewidth = 2, colour = 'black' ))
    )) +
    theme(axis.title.x = element_blank(),
        axis.text.y = element_text(size = 24, face = "bold", family = "NimbusSan"))



In [154]:
# UpSetR plots for epidemic - endemic KS data comparisons
epiend_upset_input_tibble <- alluvial_data |>
    filter(str_detect(repertoire_id, "008_021|008_057"))  |>
    left_join(study_metadata, by = "repertoire_id") |>
    dplyr::select(repertoire_id, junction_aa, tissue_type, phenotype) |>
    mutate(present = 1,
        row_id = str_c(repertoire_id, tissue_type, phenotype, sep = " ")) |>
    pivot_wider(id_cols = junction_aa, names_from = repertoire_id, values_from = present, values_fill = 0)

epiend_upset_rownames <- epiend_upset_input_tibble |>
    pull(junction_aa)
epiend_upset_input_table <- epiend_upset_input_tibble |>
    dplyr::select(-junction_aa) |>
    as.data.frame() 
rownames(epiend_upset_input_table) <- epiend_upset_rownames

epiend_upset_data <- upset_data(epiend_upset_input_table, 
    intersect = c("008_021_A", "008_021_B", "008_021_C", "008_021_D", "008_057_D", "008_057_C", "008_057_B", "008_057_A"))

combined_pal <- c("Epidemic KS - NAT" = "#fb8072", 
  "Epidemic KS - Tumor" = "#e41a1c", "Endemic KS - NAT" = "#80b1d3",
  "Endemic KS - Tumor" = "#377eb8", "Inter-subject\nNAT - NAT" = "#1b9e77", 
  "Intra-subject\nNAT - Tumor" = "#d95f02",
  "Intra-subject\nTumor - Tumor" = "#7570b3", 
  "Inter-subject\nTumor - Tumor" = "#e7298a",
  "Other" = "#606060")
epiend_pal_table <- epiend_upset_data$plot_intersections_subset |>
    as_tibble() |> 
    dplyr::rename(intersections = value) |>
    mutate(group = case_when(intersections == "008_021_A" ~ "Epidemic KS - NAT",
        intersections == "008_057_A" ~ "Endemic KS - NAT",
        intersections %in% c("008_057_B","008_057_C","008_057_D")~ "Endemic KS - Tumor",
        intersections %in% c("008_021_B","008_021_C","008_021_D")~ "Epidemic KS - Tumor",
        str_detect(intersections, "008_021") & !str_detect(intersections,"008_057") & str_detect(intersections,"A") & str_detect(intersections,'-') ~ "Intra-subject\nNAT - Tumor",
        !str_detect(intersections, "008_021") & str_detect(intersections,"008_057") & str_detect(intersections,"A") & str_detect(intersections,'[BCD]') ~ "Intra-subject\nNAT - Tumor",
        str_detect(intersections, "008_021") & !str_detect(intersections,"008_057") & !str_detect(intersections,"A") & str_detect(intersections,'[BCD]') ~ "Intra-subject\nTumor - Tumor",
        !str_detect(intersections, "008_021") & str_detect(intersections,"008_057") & !str_detect(intersections,"A") & str_detect(intersections,'[BCD]') ~ "Intra-subject\nTumor - Tumor",
        str_detect(intersections, "008_021") & str_detect(intersections,"008_057") & str_detect(intersections,"008_021_B|008_021_C|008_021_D") & str_detect(intersections,"008_057_B|008_057_C|008_057_D") ~ "Inter-subject\nTumor - Tumor",
        intersections == "008_021_A-008_057_A" ~ "Inter-subject\nNAT - NAT",
        str_detect(intersections,"008_021") & str_detect(intersections,"008_057") & !str_detect(intersections,"008_021_B") & !str_detect(intersections,"008_021_C") & !str_detect(intersections,"008_021_D") ~ "Other",
        str_detect(intersections,"008_021") & str_detect(intersections,"008_057") & !str_detect(intersections,"008_057_B") & !str_detect(intersections,"008_057_C") & !str_detect(intersections,"008_057_D") ~ "Other"),
        color = combined_pal[group])
epiend_pal <- epiend_pal_table |>
  pull(color) 
names(epiend_pal) <- epiend_pal_table |>
  pull(intersections) 

epiend_upset <- upset(epiend_upset_input_table, 
    intersect = c("008_021_A", "008_021_B", "008_021_C", "008_021_D", "008_057_D", "008_057_C", "008_057_B", "008_057_A"),
    queries = list(
        upset_query(set = '008_021_A', fill = '#fb8072'),
        upset_query(set = '008_021_B', fill = '#e41a1c'),
        upset_query(set = '008_021_C', fill = '#e41a1c'),
        upset_query(set = '008_021_D', fill = '#e41a1c'),
        upset_query(set = '008_057_A', fill = '#80b1d3'),
        upset_query(set = '008_057_B', fill = '#377eb8'),
        upset_query(set = '008_057_C', fill = '#377eb8'),
        upset_query(set = '008_057_D', fill = '#377eb8'),
        upset_query(intersect = c('008_057_A','008_057_B','008_057_C','008_057_D'), color = '#d95f02', fill = '#d95f02', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_057_B','008_057_C','008_057_D'), color = '#7570b3', fill = '#7570b3', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_057_A','008_057_C','008_057_D'), color = '#d95f02', fill = '#d95f02', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_057_C','008_057_D'), color = '#7570b3', fill = '#7570b3', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_057_A','008_057_B','008_057_D'), color = '#d95f02', fill = '#d95f02', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_057_B','008_057_D'), color = '#7570b3', fill = '#7570b3', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_057_A','008_057_D'), color = '#d95f02', fill = '#d95f02', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_057_D'), color = '#377eb8', fill = '#377eb8', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_057_A','008_057_B','008_057_C'), color = '#d95f02', fill = '#d95f02', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_057_B','008_057_C'), color = '#7570b3', fill = '#7570b3', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_057_A','008_057_C'), color = '#d95f02', fill = '#d95f02', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_057_C'), color = '#377eb8', fill = '#377eb8', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_057_A','008_057_B'), color = '#d95f02', fill = '#d95f02', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_057_B'), color = '#377eb8', fill = '#377eb8', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_057_A'), color = '#80b1d3', fill = '#80b1d3', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_057_B','008_057_C','008_057_D','008_021_D'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_057_D','008_021_D'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_057_C','008_021_D'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_057_B','008_021_D'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_057_A','008_021_D'), color = '#606060', fill = '#606060', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_021_D'), color = '#e41a1c', fill = '#e41a1c', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_057_C','008_057_D','008_021_C'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_057_A','008_057_B','008_057_D','008_021_C'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_057_B','008_057_D','008_021_C'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_057_D','008_021_C'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_057_B','008_057_C','008_021_C'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_057_C','008_021_C'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_057_B','008_021_C'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_057_A','008_021_C'), color = '#606060', fill = '#606060', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_057_B','008_057_D','008_021_D','008_021_C'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_057_D','008_021_D','008_021_C'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_021_D','008_021_C'), color = '#7570b3', fill = '#7570b3', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_021_C'), color = '#e41a1c', fill = '#e41a1c', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_057_B','008_057_D','008_021_B'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_057_D','008_021_B'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_057_B','008_021_B'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_057_C','008_021_D','008_021_B'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_021_D','008_021_B'), color = '#7570b3', fill = '#7570b3', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_057_A','008_021_C','008_021_B'), color = '#606060', fill = '#606060', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_021_D','008_021_C','008_021_B'), color = '#7570b3', fill = '#7570b3', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_021_C','008_021_B'), color = '#7570b3', fill = '#7570b3', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_021_B'), color = '#e41a1c', fill = '#e41a1c', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_057_D','008_021_A'), color = '#606060', fill = '#606060', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_021_D','008_021_A'), color = '#d95f02', fill = '#d95f02', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_021_D','008_021_C','008_021_A'), color = '#d95f02', fill = '#d95f02', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_021_C','008_021_A'), color = '#d95f02', fill = '#d95f02', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_021_D','008_021_B','008_021_A'), color = '#d95f02', fill = '#d95f02', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_057_A','008_057_B','008_057_C','008_057_D','008_021_D','008_021_C','008_021_B','008_021_A'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_057_A','008_057_B','008_057_D','008_021_D','008_021_C','008_021_B','008_021_A'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_057_D','008_021_D','008_021_C','008_021_B','008_021_A'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_057_B','008_021_D','008_021_C','008_021_B','008_021_A'), color = '#e7298a', fill = '#e7298a', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_021_D','008_021_C','008_021_B','008_021_A'), color = '#d95f02', fill = '#d95f02', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_021_C','008_021_B','008_021_A'), color = '#d95f02', fill = '#d95f02', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_021_B','008_021_A'), color = '#d95f02', fill = '#d95f02', only_components=c('intersections_matrix', 'Intersection size')),
        upset_query(intersect = c('008_021_A'), color = '#fb8072', fill = '#fb8072', only_components=c('intersections_matrix', 'Intersection size'))),
    intersections = list(
        '008_057_B','008_057_D','008_057_C','008_057_A',
        '008_021_C','008_021_D','008_021_B','008_021_A',
        c('008_057_B','008_057_D'),
        c('008_057_B','008_057_C','008_057_D'),
        c('008_057_B','008_057_C'),
        c('008_057_C','008_057_D'),
        c('008_021_D','008_021_C'),
        c('008_021_D','008_021_C','008_021_B'),
        c('008_021_C','008_021_B'),
        c('008_021_D','008_021_B'),
        c('008_057_A','008_057_B','008_057_C','008_057_D'),
        c('008_057_A','008_057_D'),
        c('008_057_A','008_057_B'),
        c('008_057_A','008_057_C'),
        c('008_057_A','008_057_B','008_057_D'),
        c('008_057_A','008_057_C','008_057_D'),
        c('008_057_A','008_057_B','008_057_C'),
        c('008_021_D','008_021_C','008_021_B','008_021_A'),
        c('008_021_D','008_021_C','008_021_A'),
        c('008_021_B','008_021_A'),
        c('008_021_C','008_021_A'),
        c('008_021_C','008_021_B','008_021_A'),
        c('008_021_D','008_021_A'),
        c('008_021_D','008_021_B','008_021_A'),
        c('008_057_B','008_057_D','008_021_C'),
        c('008_057_B','008_021_C'),
        c('008_057_D','008_021_D'),
        c('008_057_B','008_021_D'),
        c('008_057_D','008_021_C'),
        c('008_057_D','008_021_B'),
        c('008_057_B','008_021_B'),
        c('008_057_B','008_021_D','008_021_C','008_021_B','008_021_A'),
        c('008_057_B','008_057_C','008_057_D','008_021_D'),
        c('008_057_C','008_021_D'),
        c('008_057_C','008_057_D','008_021_C'),
        c('008_057_A','008_057_B','008_057_D','008_021_C'),
        c('008_057_B','008_057_C','008_021_C'),
        c('008_057_C','008_021_C'),
        c('008_057_B','008_057_D','008_021_D','008_021_C'),
        c('008_057_D','008_021_D','008_021_C'),
        c('008_057_B','008_057_D','008_021_B'),
        c('008_057_C','008_021_D','008_021_B'),
        c('008_057_A','008_057_B','008_057_C','008_057_D','008_021_D','008_021_C','008_021_B','008_021_A'),
        c('008_057_A','008_057_B','008_057_D','008_021_D','008_021_C','008_021_B','008_021_A'),
        c('008_057_D','008_021_D','008_021_C','008_021_B','008_021_A'),
        c('008_057_A','008_021_D'),
        c('008_057_A','008_021_C'),
        c('008_057_A','008_021_C','008_021_B'),
        c('008_057_D','008_021_A')

    ),
    sort_sets=FALSE,
    sort_intersections=FALSE,
    width_ratio=0.1,
    name = "Overlap between TCR repertoires",
    base_annotations = list(
            'Intersection size'=intersection_size(
                aes(fill = intersection),
                text = list(size = 7, angle = 90, hjust = 0, vjust = 0.5)
            ) + ylab('Number of overlapping TCRs') 
            + theme_classic() 
            + ggtitle("Endemic KS - Epidemic KS") 
            + labs(tag = "C") 
            + scale_fill_manual(values = epiend_pal)
            + theme(axis.title.x = element_blank(),
                plot.title = element_text(size = 30, face = "bold", family = "NimbusSan", hjust = 0.5),
                axis.text.x = element_blank(),
                axis.ticks.x = element_blank(),
                axis.title.y = element_text(size = 24, face = "bold", family = "NimbusSan"),
                axis.text.y = element_text(size = 22, face = "bold", family = "NimbusSan"),
                plot.tag = element_text(size = 28, face = "bold", family = "NimbusSan"),
                legend.position = "right")),
    set_sizes=(upset_set_size()+ ylab('Number of\nunique TCRs') 
            + theme_void() 
            + theme(axis.title.y = element_blank(),
                axis.text.y = element_blank(),
                axis.ticks.y = element_blank(),
                axis.title.x = element_text(size = 24, face = "bold", family = "NimbusSan"),
                axis.text.x = element_text(size = 20, face = "bold", family = "NimbusSan", angle = 90, vjust = 0.5),
                axis.ticks.x = element_line(linewidth = 2, colour = 'black' ))
    ), guides = "collect") +
    scale_fill_manual(values = epiend_pal) +
    theme(axis.title.x = element_text(size = 24, face = "bold", family = "NimbusSan"),
        axis.text.y = element_text(size = 24, face = "bold", family = "NimbusSan")) 


In [155]:
figure_five <- endemic_upset / epidemic_upset / epiend_upset + 
    plot_layout(guides = "collect")  & 
    theme(text = element_text('NimbusSan'))

ggsave(str_c(figures_path, "pdf", "Figure_five.pdf", sep = "/"), figure_five, width = 32, 
    height = 30, device = cairo_pdf, family = "Arial Unicode MS")
ggsave(str_c(figures_path, "png", "Figure_five.png", sep = "/"), figure_five, width = 32, 
    height = 30)
ggsave(str_c(figures_path, "svg", "Figure_five.svg", sep = "/"), figure_five, width = 32, 
    height = 30)


# T-cells from clustered unknown antigenic specificity groups persist in KS tumors across time

## Figure 6: T-cells from GLIPH2-defined groups with specificity for unknown antigens persist in the TME across time and space in individuals with epidemic and endemic KS.

In [156]:
temopral_example_table <- study_annotated_nprod_table |>
  filter(patient_id %in% c("008_141", "008_008")) |>
  productiveSeq(aggregate = "junction_aa") 

temporal_overlap_samples <- study_metadata |> 
  filter(tissue_type %in% c( "Tumor") & 
    str_detect(repertoire_id, "008_\\d+")) |>  
  group_by(patient_id, hiv_status, visit_code) |> 
  summarize(count = length(unique(repertoire_id))) |> 
  pivot_wider(names_from = visit_code, values_from = count, values_fill = 0) |> 
  filter(V01 != 0 & (V05 != 0 | V08 != 0)) |>
  pull(patient_id)

visit_map <- study_metadata |>
  filter(tissue_type %in% c( "Tumor") & 
    str_detect(repertoire_id, "008_\\d+")) |>
  pull(visit_code)
names(visit_map) <- study_metadata |>
  filter(tissue_type %in% c( "Tumor") & 
    str_detect(repertoire_id, "008_\\d+")) |>
  pull(trb_repertoire_id)

endemic_ug <- temopral_example_table |> 
  filter(junction_aa %in% kstme_clustered_unknown_table$junction_aa & 
    str_detect(repertoire_id, "008_141")) |>
  mutate(color = "#377eb8") |>
  pull(color) 

endemic_alist <- temopral_example_table |> 
  filter(junction_aa %in% kstme_clustered_unknown_table$junction_aa & 
    str_detect(repertoire_id, "008_141")) |>
  mutate(color = "#377eb8") |>
  pull(junction_aa) 
names(endemic_ug) <- temopral_example_table |> 
  filter(junction_aa %in% kstme_clustered_unknown_table$junction_aa & 
    str_detect(repertoire_id, "008_141")) |>
  mutate(color = "#377eb8") |>
  pull(junction_aa) 

endemic_alluvial <- temopral_example_table |>
  filter(str_detect(repertoire_id, "008_141")) |>
  topSeqs(top = 500) |>
  cloneTrack() |>
  plotTrack(alist = endemic_alist, apal = endemic_ug) +
  scale_x_discrete(labels = temporal_map_dict) +
  labs(x = "Sample") + 
  theme_classic(base_size = 30) +
  theme(legend.position = "none", 
    axis.text.x = element_text(face = "bold", color = "black"))

epidemic_ug <- temopral_example_table |> 
  filter(junction_aa %in% kstme_clustered_unknown_table$junction_aa & 
    str_detect(repertoire_id, "008_008")) |>
  mutate(color = "#e41a1c") |>
  pull(color) 
epidemic_alist <- temopral_example_table |> 
  filter(junction_aa %in% kstme_clustered_unknown_table$junction_aa & 
    str_detect(repertoire_id, "008_008")) |>
  mutate(color = "#e41a1c") |>
  pull(junction_aa) 
names(epidemic_ug) <- temopral_example_table |> 
  filter(junction_aa %in% kstme_clustered_unknown_table$junction_aa & 
    str_detect(repertoire_id, "008_008")) |>
  mutate(color = "#e41a1c") |>
  pull(junction_aa) 

epidemic_alluvial <- temopral_example_table |>
  filter(str_detect(repertoire_id, "008_008")) |>
  topSeqs(top = 500) |>
  cloneTrack() |>
  plotTrack(alist = epidemic_alist, apal = epidemic_ug) +
  scale_x_discrete(labels = temporal_map_dict) +
  labs(x = "Sample") + 
  theme_classic(base_size = 30)  +
  theme(legend.position = "none",
   axis.text.x = element_text(face = "bold", color = "black"))

figure_six_layout <- "
11112222
11112222
11112222"
figure_six <- (endemic_alluvial + epidemic_alluvial +plot_layout(guides = "auto"))  +
  plot_layout(design = figure_six_layout) +
  plot_annotation(tag_levels = 'A') & theme(text = element_text('NimbusSan', face = "bold"))
ggsave(str_c(figures_path, "pdf", "Figure_six.pdf", sep = "/"), figure_six, 
  width = 30, height = 30)
ggsave(str_c(figures_path, "svg", "Figure_six.svg", sep = "/"), figure_six, 
  width = 30, height = 30)
ggsave(str_c(figures_path, "png", "Figure_six.png", sep = "/"), figure_six, 
  width =  30, height = 30)


Subsetting productive sequences [=>----------]  14% eta:  2s

Subsetting productive sequences [==>---------]  21% eta:  3s

Subsetting productive sequences [==>---------]  29% eta:  3s

Subsetting productive sequences [===>--------]  36% eta:  3s

Subsetting productive sequences [====>-------]  43% eta:  7s

Subsetting productive sequences [=====>------]  50% eta:  5s

Subsetting productive sequences [======>-----]  57% eta:  4s

Subsetting productive sequences [=======>----]  64% eta:  3s

Subsetting productive sequences [========>---]  71% eta:  3s

Subsetting productive sequences [========>---]  79% eta:  3s

Subsetting productive sequences [=========>--]  86% eta:  2s

Subsetting productive sequences [==========>-]  93% eta:  1s

Subsetting productive sequences [============] 100% eta:  0s




ERROR: Error in eval(expr, envir, enclos): object 'temporal_map_dict' not found
